In [ ]:
%%capture
!pip install torch-geometric==2.5.0
!pip install trimesh
!pip install differentiable-voronoi
!pip install gradnorm-pytorch

In [ ]:
try:
  import pydmd as dmd
except ImportError:
  !pip install pydmd==<2025.5.1>
  import pydmd as dmd
import time

In [ ]:
from differentiable_voronoi import differentiable_voronoi, triangulate
from shapely import Polygon, LineString, Point
import matplotlib.pyplot as plt
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import to_dense_adj, to_undirected, degree, to_dense_batch
from copy import deepcopy
import numpy as np
from sklearn.cluster import KMeans, SpectralClustering, BisectingKMeans
from trimesh import Trimesh
from scipy.spatial import Delaunay
import matplotlib.patches
from shapely.geometry.polygon import orient
from scipy.spatial import Voronoi, voronoi_plot_2d
from torch_geometric.nn.pool import avg_pool_x
import copy
from torch.nn import MSELoss
from IPython.display import clear_output
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KDTree
from time import sleep
from typing import List, Optional, Tuple, Union
from torch_geometric.nn import GCNConv, DenseGraphConv, GraphConv, GATConv
import torch.nn.functional as F
from torch import Tensor
from torch_geometric.nn.dense.mincut_pool import _rank3_trace
from torch.nn.functional import cosine_similarity
import torch.nn.utils as nn_utils
from torch_geometric.nn.models.mlp import MLP
from gradnorm_pytorch import GradNormLossWeighter


In [ ]:
# these functions are needed for slightly modified differentiable_voronoi function needed for calculating boundary nodes

from differentiable_voronoi.differentiable_voronoi import center_of_circumcircle, complete_boundary_cells, orient_vertex_faces_dict, crop_cells, select_edges_adjacent_to_the_given_vetrices, calc_lengths_of_voronoi_edges_adjacent_to_vertices_to_crop, calc_lengths_of_inner_voronoi_edges, calc_areas_dict_bounded, draw_voronoi_region

In [ ]:
# modified differentiable voronoi function to obtain boundary nodes

def differentiable_voronoi(coords, mesh, boundary=None, vizualize=False):
    vertex_faces_dict = {}
    for i, row in enumerate(mesh.vertex_faces):
        vertex_faces_dict[i] = [face for face in row if face != -1]

    assert len(vertex_faces_dict) == len(coords), print(len(vertex_faces_dict), len(coords))

    circumcenters = center_of_circumcircle(coords, torch.tensor(mesh.faces))
    clipped_vertices_dict = None

    circumcenters, vertex_faces_dict = complete_boundary_cells(coords, vertex_faces_dict, circumcenters,
                                                               mesh.boundary_nodes, mesh)
    assert len(vertex_faces_dict) == len(coords), print(len(vertex_faces_dict), len(coords))
    circumcenters, vertex_faces_dict = orient_vertex_faces_dict(coords, vertex_faces_dict, circumcenters,
                                                                mesh.boundary_nodes)
    assert len(vertex_faces_dict) == len(coords)

    if boundary is not None:
        clipped_vertices_dict, voronoi_region_to_keep, voronoi_regions_to_crop = crop_cells(coords, vertex_faces_dict,
                                                                                            circumcenters, boundary)
        assert len(clipped_vertices_dict) == len(coords)
        edges_adjacent_to_vertices_to_crop = select_edges_adjacent_to_the_given_vetrices(voronoi_regions_to_crop,
                                                                                         mesh.edges_unique)
        lengths_of_voronoi_edges, edges_to_delete = calc_lengths_of_voronoi_edges_adjacent_to_vertices_to_crop(
            edges_adjacent_to_vertices_to_crop,
            clipped_vertices_dict,
            mesh.edges_unique)

        lengths_of_voronoi_edges = calc_lengths_of_inner_voronoi_edges(
            set(list(range(len(mesh.edges_unique)))) - edges_adjacent_to_vertices_to_crop,
            circumcenters,
            lengths_of_voronoi_edges,
            mesh.edges_unique_faces)
        lengths_of_voronoi_edges = lengths_of_voronoi_edges[~edges_to_delete]
        areas = calc_areas_dict_bounded(clipped_vertices_dict)
    else:
        edges_to_delete = np.full(len(mesh.edges_unique), False)
        areas = calc_areas_dict(circumcenters, vertex_faces_dict)
        lengths_of_voronoi_edges = torch.zeros((len(mesh.edges_unique)))
        lengths_of_voronoi_edges = calc_lengths_of_inner_voronoi_edges(set(list(range(len(mesh.edges_unique)))),
                                                                       circumcenters,
                                                                       lengths_of_voronoi_edges,
                                                                       mesh.edges_unique_faces)

    assert len(mesh.edges_unique[~edges_to_delete]) == len(lengths_of_voronoi_edges)
    assert len(areas[areas == 0]) == 0

    if vizualize:
        with torch.no_grad():
            fig = plt.figure(figsize=(10, 10))
            ax = fig.add_subplot()
            if boundary is not None:
                for vertex in voronoi_region_to_keep:
                    line = clipped_vertices_dict[vertex]
                    draw_voronoi_region(coords, line, vertex, ax)
                for vertex in voronoi_regions_to_crop:
                    line = clipped_vertices_dict[vertex]
                    draw_voronoi_region(coords, line, vertex, ax, color='r')
            else:
                for vertex in vertex_faces_dict.keys():
                    line = circumcenters[vertex_faces_dict[vertex]].clone().detach()
                    draw_voronoi_region(coords, line, vertex, ax)
            plt.show()

    return torch.tensor(mesh.edges_unique[~edges_to_delete].T), areas, lengths_of_voronoi_edges, clipped_vertices_dict, voronoi_regions_to_crop

## Solvers

In [ ]:
class Solver(MessagePassing):
    """Graph solver class for parabolic equation"""
    def __init__(self):
        super().__init__(aggr='add', flow='target_to_source')

    def calc_weights(self, k, e, h, edge_index):
        k_i, k_j = k[edge_index[0]], k[edge_index[1]]
        numerator = 2 * k_i * k_j * e
        denominator = (k_i + k_j) * h + 1e-8
        assert numerator.shape == denominator.shape
        return torch.divide(numerator, denominator)

    def forward(self, u, v, h, e, k, w, p_bhp, dt, edge_index):
        assert ~torch.any(torch.isnan(u))
        assert ~torch.any(torch.isnan(v))
        assert ~torch.any(torch.isnan(h))
        assert ~torch.any(torch.isnan(e))
        assert ~torch.any(torch.isnan(k))
        assert len(u) == len(v), print(len(u), len(v))
        assert len(h) == len(e) == edge_index.shape[1]
        assert len(u) == len(p_bhp) == len(w) == len(k)
        assert torch.all(u >= 0)
        assert torch.all(v > 0)
        assert torch.all(h > 0)
        assert torch.all(e >= 0)
        assert torch.all(k > 0)
        assert torch.all(w >= 0)
        assert torch.all(p_bhp >= 0)


        weights = self.calc_weights(k, e, h, edge_index).view(-1, 1)
        assert ~torch.any(torch.isnan(weights))
        assert len(weights) == edge_index.shape[1]#, print(len(weights), len(edge_index))
        edge_index, weights = to_undirected(edge_index, weights)
        torch.save(edge_index, 'edge_index.pt')
        assert torch.all(weights >= 0)
        div = self.propagate(edge_index=edge_index, x=u, weights=weights)
        assert len(div) == len(u)
        t_over_V = dt / v
        numerator = u + t_over_V * (w * p_bhp - div)
        denominator = 1 + t_over_V * w
        return numerator / denominator

    def message(self, x_i, x_j, weights):
        du = x_i - x_j
        return weights * du

def calc_h(coords, edge_index):
    x_i, x_j = coords[edge_index[0]], coords[edge_index[1]]
    return torch.norm(x_i - x_j, dim=1)

def roll_out(solver, n_steps, u0, v, h, e, k, w, p_bhp, dt, edge_index, keep=0, save_animation=False, every_nth=10):
    roll = torch.zeros((len(keep), n_steps))
    animation=[]
    u = u0
    for i in range(n_steps):
        u = solver(u=u, v=v.view(-1, 1), h=h, e=e, k=k, w=w, p_bhp=p_bhp, dt=dt, edge_index=edge_index)
        roll[:, i] = u[keep].squeeze()
        if save_animation and i%every_nth==0: animation.append(u.clone().detach())
    return roll, animation

def dynamics(point_cloud, sources, sinks, boundary, dt=0.0001, n_steps=10000, save_animation=False, every_nth=10):
    COORDS = torch.tensor(point_cloud[:, :2], dtype=torch.float32, requires_grad=False)
    mesh = triangulate(COORDS.clone().detach())

    edge_index, areas, e, _,_ = differentiable_voronoi(COORDS, mesh, boundary)
    assert len(COORDS) == len(areas), print(len(COORDS), len(areas), areas)
    h = calc_h(COORDS, edge_index)

    u0 = torch.zeros((len(COORDS), 1), requires_grad=False)
    perm = torch.tensor(point_cloud[:, 2], requires_grad=False)

    p_bhp = torch.zeros((len(COORDS), 1), requires_grad=False)
    w = torch.zeros((len(COORDS), 1), requires_grad=False)
    for s, power in sources.items():
        p_bhp[s] = power
        w[s] = 1

    return roll_out(Solver(), n_steps=n_steps, u0=u0, v=areas, h=h, e=e, k=perm, w=w, p_bhp=p_bhp, dt=dt,
                                 edge_index=edge_index, keep=sinks, save_animation=save_animation, every_nth=every_nth)

In [ ]:
def roll_out_1(solver, n_steps, u0, v, h, e, k, w, p_bhp, dt, edge_index, keep=0, save_animation=False, every_nth=10):
    roll = torch.zeros((len(u0), n_steps))
    animation=[]
    u = u0
    for i in range(n_steps):

        u = solver(u=u, v=v.view(-1, 1), h=h, e=e, k=k, w=w, p_bhp=p_bhp, dt=dt, edge_index=edge_index)
        roll[:, i] = u.squeeze()
        if save_animation and i%every_nth==0: animation.append(u.clone().detach())
    return roll, animation

def dynamics_1(point_cloud, sources, sinks, boundary, dt=0.0001, n_steps=10000, save_animation=False, every_nth=10):
    COORDS = torch.tensor(point_cloud[:, :2], dtype=torch.float32, requires_grad=False)
    mesh = triangulate(COORDS.clone().detach())

    edge_index, areas, e, _,_ = differentiable_voronoi(COORDS, mesh, boundary)
    assert len(COORDS) == len(areas), print(len(COORDS), len(areas), areas)
    h = calc_h(COORDS, edge_index)

    u0 = torch.zeros((len(COORDS), 1), requires_grad=False)
    perm = torch.tensor(point_cloud[:, 2], requires_grad=False)

    p_bhp = torch.zeros((len(COORDS), 1), requires_grad=False)
    w = torch.zeros((len(COORDS), 1), requires_grad=False)
    for s, power in sources.items():
        p_bhp[s] = power
        w[s] = 1

    return roll_out_1(Solver(), n_steps=n_steps, u0=u0, v=areas, h=h, e=e, k=perm, w=w, p_bhp=p_bhp, dt=dt,
                                 edge_index=edge_index, keep=sinks, save_animation=save_animation, every_nth=every_nth)

## Learnable coarsener

In [ ]:
EPS = 1e-15  # Small value to avoid division by zero WITHOUT VOXELIZATION BUT WITH EDGE_WEIGHTS/SOURCES/SINKS


class learnableCoarsening(torch.nn.Module):
    def __init__(self,
                 channels: Union[int, List[int]],
                 k: int,
                 sources: List[int],
                 sinks: List[int],
                 boundary_nodes: List[int],
                 dropout: float = 0.0,
                 hidden_channels: int = 3,
                 denseconv = False,
                 num_layers: int=1,
                 temperature: float=1.0,
                 gatconv = True,
                 seed: int=42
                 ):
        super().__init__()

        if isinstance(channels, int):
            channels = [channels]
        self.temperature=temperature
        self.convs = torch.nn.ModuleList()
        self.input_dim = channels[0] + 3
        #self.mlp = MLP(channels  + [k], act="elu", norm=None)
        #self.mlp = MLP([self.input_dim] + channels[1:] + [k], act="elu", norm=None)
        self.mlp = MLP([hidden_channels, k], act="elu", norm=None)
        for i in range(num_layers):
            in_dim = self.input_dim if i == 0 else hidden_channels
            if denseconv:
                self.convs.append(DenseGraphConv(in_dim, hidden_channels))
                #self.convs.append(GraphConv(in_dim, hidden_channels))
            else:
                if gatconv:
                    self.convs.append(GATConv(in_dim, hidden_channels, edge_dim=1))
                else:
                    self.convs.append(GraphConv(in_dim, hidden_channels))


        self.denseconv=denseconv
        self.dropout = dropout

        self.reset_parameters(seed)
        self.intensities = sources
        self.sources = torch.tensor(list(sources.keys()), dtype=torch.int)
        self.sinks = torch.tensor(sinks, dtype=torch.int)
        self.boundary = boundary_nodes
        #self.temperature = torch.nn.Parameter(torch.tensor(temperature))



    def reset_parameters(self,seed):
        r"""Resets all learnable parameters of the module."""
        torch.manual_seed(seed)
        self.mlp.reset_parameters()
        # Custom initialization of layer weights
        for conv in self.convs:
            for name, param in conv.named_parameters():
                if 'weight' in name:
                    torch.nn.init.xavier_uniform_(param,gain=1.0)
                    #torch.nn.init.orthogonal_(param)
                    #torch.nn.init.normal_(param, mean=0.0, std=1.0)

                elif 'bias' in name:
                    #torch.nn.init.zeros_(param)
                    torch.nn.init.normal_(param, mean=0.0, std=0.01)


        for name, param in self.mlp.named_parameters():
            if 'weight' in name:
                torch.nn.init.xavier_uniform_(param,gain=2.0)
                #torch.nn.init.normal_(param, mean=0.0, std=1.0)

            elif 'bias' in name:
                #torch.nn.init.zeros_(param)
                torch.nn.init.normal_(param, mean=0.0, std=0.01)



    def forward(
        self,
        x: Tensor,
        adj: Tensor,
        edge_weights: Tensor
    ) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor, Tensor]:
        r"""Forward pass.

        Args:
            x (torch.Tensor): Node feature tensor
                :math:`\mathbf{X} \in \mathbb{R}^{B \times N \times F}`, with
                batch-size :math:`B`, (maximum) number of nodes :math:`N` for
                each graph, and feature dimension :math:`F`.
                Note that the cluster assignment matrix
                :math:`\mathbf{S} \in \mathbb{R}^{B \times N \times C}` is
                being created within this method.
            adj (torch.Tensor): Adjacency tensor

        :rtype: (:class:`torch.Tensor`, :class:`torch.Tensor`,
            :class:`torch.Tensor`, :class:`torch.Tensor`,
        """
        x = x.unsqueeze(0) if x.dim() == 2 else x
        B, N, F = x.size()

        # Create masks for sources and sinks
        device = x.device
        source_mask = torch.zeros(N, 1, device=device) + 1e-8
        sink_mask   = torch.zeros(N, 1, device=device) + 1e-8
        boundary_mask = torch.zeros(N, 1, device=device) + 1e-8

        for key, power in self.intensities.items():
            source_mask[key] = power
        sink_mask[self.sinks] = 1.0
        boundary_mask[self.boundary] = 1.0

        # Expand to batch and concatenate with x
        source_mask = source_mask.unsqueeze(0).expand(B, -1, -1)
        sink_mask = sink_mask.unsqueeze(0).expand(B, -1, -1)
        boundary_mask = boundary_mask.unsqueeze(0).expand(B, -1, -1)
        x_aug = torch.cat([x, source_mask, sink_mask,boundary_mask], dim=-1)

        #x_aug = torch.cat([x, source_mask, sink_mask], dim=-1)

        conv_matrix=x_aug[0]
        #print("x dim:", x.dim())
        #print(x.shape)
        for conv in self.convs:
            # edge_weights = edge_weights.view(-1, 1)
            # conv_matrix=torch.nn.functional.elu(conv(conv_matrix, adj, edge_weights))
            if self.denseconv:

                conv_matrix = torch.nn.functional.elu(conv(conv_matrix, adj))
            else:
                edge_weights = edge_weights.view(-1, 1)
                conv_matrix = torch.nn.functional.elu(conv(conv_matrix, adj, edge_weights))

        s = self.mlp(conv_matrix)
        s = s.unsqueeze(0) if s.dim() == 2 else s
        #print(s.dim())
        s_max = torch.max(s, dim=-1, keepdim=True)[0]  # Max value along the last dimension
        s = s - s_max  # Subtract the max value for numerical stability

        #print("T: ", self.temperature)

        s = torch.softmax(s/self.temperature, dim=-1)
        sources = torch.zeros((s.shape[1], len(self.sources)))
        sources[self.sources, torch.arange(len(self.sources))] = 1.
        sinks = torch.zeros((s.shape[1], len(self.sinks)))
        sinks[self.sinks, torch.arange(len(self.sinks))] = 1.
        s = torch.cat([s, sinks.unsqueeze(0)], dim=-1)
        s = torch.cat([s, sources.unsqueeze(0)], dim=-1)

        (batch_size, num_nodes, _), C = x.size(), s.size(-1)
        '''
        s_normed = s.transpose(1, 2) / (torch.sum(s.transpose(1, 2), dim=2, keepdim=True))
        out = torch.matmul(s_normed, x)
        '''
        s_normed = s.transpose(1, 2) / (torch.sum(s.transpose(1, 2), dim=2, keepdim=True))
        aggregated = torch.matmul(s_normed, x_aug)  # shape: [B, k, F+2]

        out = aggregated


        return None, out,None

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.mlp.in_channels}, '
                f'num_clusters={self.mlp.out_channels})')

## Stability criterion check and stability loss


In [ ]:


class StabilityCriterionCheck(MessagePassing):
    r"""
    Calculates the real stable time step for the current configuration
    of a parabolic equation based on graph-based propagation.

    Args:
        v (Tensor): cell areas tensor.
        k (Tensor): permeability tensor.
        e (Tensor): Length of the boundary between Voronoi regions (length of the edge).
        h (Tensor): Distance between the centers of Voronoi regions.
        edge_index (Tensor): Graph edge indices.

    Returns:
        float: The computed stable time step.
    """


    def __init__(self):
        super().__init__(aggr='max', flow='target_to_source')

    def forward(self, v, k, e, h, edge_index):
        v_min = min(v)
        k_max = max(k)
        C = self.propagate(edge_index=edge_index, e=e, h=h)
        C_max = max(C)
        return (2 * v_min) / (C_max * k_max)

    def message(self, e, h):
        return e / (h + 1e-8)



class StabilityCriterion(MessagePassing):
    r"""
    Calculates the stability loss for the current configuration
    of a parabolic equation based on graph-based propagation.

    Args:
        v (Tensor): cell areas tensor.
        k (Tensor): permeability tensor.
        e (Tensor): Length of the boundary between Voronoi regions (length of the edge).
        h (Tensor): Distance between the centers of Voronoi regions.
        edge_index (Tensor): Graph edge indices.

    Returns:
        float: The computed stability loss.
    """
    @staticmethod
    def smooth_max_min(x,alpha=10.0):
        weights = torch.softmax(alpha * x,dim=0)
        return (x * weights).sum(dim=0)
    def __init__(self, n, s = 1.0):
        super().__init__(aggr='max', flow='target_to_source')
        self.n = n
        self.s = s

    def forward(self, v, k, e, h, edge_index):
        v_min=self.smooth_max_min(v,alpha=-1.0)
        C = self.propagate(edge_index=edge_index, e=e, h=h)
        #C=C.squeeze()
        C_max=self.smooth_max_min(C,alpha=1.0)
        #print(v_min)
        #print(C_max)
        #return -(4 / (C_max+1e-8)) * (self.n * v_min) / self.s
        return -(torch.log(4.0 * self.n * v_min/self.s) - torch.log(C_max))

        # out=-4 * (self.n * v_min) / self.s
        # out=out.unsqueeze(dim=-1)
        # return out

    def message(self, e, h):
        return e/(h + 1e-8)

In [ ]:
def ICML_print_grid(ax, COORDS, perm, sources, sinks, clipped_vertices_dict, unit='$K(x, y)$'):
    for vertex in clipped_vertices_dict.keys():
        line = clipped_vertices_dict[vertex].clone().detach()
        cmap = plt.get_cmap('RdYlBu')
        p = matplotlib.patches.Polygon(line, facecolor = 'g', edgecolor='black', alpha=0.2, fill=False)
        m = torch.mean(torch.unique(line, dim=0), dim=0)
        ax.add_patch(p)
    scale = ax.scatter(COORDS.detach()[:, 0], COORDS.detach()[:, 1], c=perm.detach(), s=400)
    if sources is not None:
        for i, src in enumerate(sources):
            ax.scatter(COORDS.detach()[src, 0].item(), COORDS.detach()[src, 1].item(), s=100, c='r', marker='v')
            ax.text(COORDS.detach()[src, 0].item(), COORDS.detach()[src, 1].item(), 'source ' + str(i+1), fontsize=30)

    if sinks is not None:
        for i, snk in enumerate(sinks):
            ax.scatter(COORDS.detach()[snk, 0].item(), COORDS.detach()[snk, 1].item(), s=100, c='g', marker='^')
            ax.text(COORDS.detach()[snk, 0].item(), COORDS.detach()[snk, 1].item(), 'sensor ' + str(i+1), fontsize=30)

    cbar = plt.colorbar(scale)
    cbar.ax.tick_params(labelsize=24)
    cbar.ax.set_ylabel(unit, fontsize=40)

def ICML_print(COORDS, clipped_vertices_dict, perm, ground_truth_dynamics, reduced_dynamics, loss_curve,file_title='', title='', sources=None, sinks=None, i=0,save_ani=False,n_steps=1000):
    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[:, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 1])

    ICML_print_grid(ax1, COORDS, perm, sources, sinks, clipped_vertices_dict)

    if reduced_dynamics is not None:
        for pp in range(len(sinks)):
            ax2.plot(reduced_dynamics[pp], label='reduced_sensor '+str(pp+1))
            ax2.plot(ground_truth_dynamics[pp], label='gt_sensor '+str(pp+1))
    #ax2.set_title('$p(t)$ at sensor')
    ax2.set_xlabel('time step')
    ax2.set_ylabel('$p_{s},~Pa$')
    ax2.legend()

    label_phys = 'phys_loss'
    ax3.plot(loss_curve,label = label_phys)
    ax3.legend()
    ax3.set_yscale('log')
    ax3.set_title('Loss')
    ax3.set_xlabel('epoch')
    ax3.set_ylabel('Loss')
    ax1.set_title(title, fontsize=24)
    if save_ani:
        plt.savefig(f'/content/ani/{str(i).zfill(10)}')
    plt.show()

In [ ]:
import random
#import neptune
def set_seeds(seed=42):
    random.seed(seed)  # Random generator python
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch
    torch.cuda.manual_seed_all(seed)  # For all GPU
    torch.backends.cudnn.deterministic = True  # Deterministic mode
    torch.backends.cudnn.benchmark = False


In [ ]:
def print_with_neptune(out,dmon, clipped_vertices_dict, point_cloud, boundary,ground_truth_dynamics, reduced_dynamics, adjacency,
                            loss_curve,phys_loss_values,stability_loss_values,s, title='', sources=None, sinks=None,run_neptune=False,
                            dt=0.001, n_steps=10000, n_epochs=100, clusters=30, channels = [3,8], lr = 0.03,
                            warmup_steps = 50, phys_loss_weight = 50,sigmoid_weight=30,savename='test_run',seed=42,unit='$K(x, y)$',i=0,api_token="",phys_start=50,project_name="",num_layers=1,temperature=1.0,denseconv=False):
    sources_reduced= list(range(len(out[0]) - len(sources),  len(out[0])))
    sinks_reduced= list(range(len(out[0]) - len(sources) - len(sinks),  len(out[0]) - len(sources)))
    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[:, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 1])



    for vertex in clipped_vertices_dict.keys():
        line = clipped_vertices_dict[vertex].clone().detach()
        cmap = plt.get_cmap('RdYlBu')
        p = matplotlib.patches.Polygon(line, facecolor = 'g', edgecolor='black', alpha=0.2, fill=False)
        m = torch.mean(torch.unique(line, dim=0), dim=0)
        ax1.add_patch(p)
    scale = ax1.scatter(out[0].detach()[:, 0], out[0].detach()[:, 1], c=out[0].detach().numpy()[:, 2], s=400)
    if sources is not None:
        for j, src in enumerate(sources_reduced):
            ax1.scatter(out[0].detach()[src, 0].item(), out[0].detach()[src, 1].item(), s=100, c='r', marker='v')
            ax1.text(out[0].detach()[src, 0].item(), out[0].detach()[src, 1].item(), 'source ' + str(j+1), fontsize=30)

    if sinks is not None:
        for j, snk in enumerate(sinks_reduced):
            ax1.scatter(out[0].detach()[snk, 0].item(), out[0].detach()[snk, 1].item(), s=100, c='g', marker='^')
            ax1.text(out[0].detach()[snk, 0].item(), out[0].detach()[snk, 1].item(), 'sink ' + str(j+1), fontsize=30)

    cbar = plt.colorbar(scale)
    cbar.ax.tick_params(labelsize=24)
    cbar.ax.set_ylabel('$K(x, y)$', fontsize=40)
    if reduced_dynamics is not None:
        for j in range(len(sinks)):
            ax2.plot(reduced_dynamics[j,:n_steps], label='reduced_sink '+str(j+1))
            ax2.plot(ground_truth_dynamics[j,:n_steps], label='gt_sink '+str(j+1))
    ax2.set_title('$u(t)$ at sinks')
    ax2.set_xlabel('time step')
    ax2.legend()


    label_stability = 'stability loss'
    ax3.plot(stability_loss_values, label=label_stability)
    if phys_loss_values is not None:
        label_phys = 'phys loss'
        ax3.plot(list(range(phys_start, phys_start + len(phys_loss_values))), phys_loss_values, label=label_phys)

    ax3.legend()
    ax3.set_yscale('log')
    ax3.set_title('Loss')
    ax3.set_xlabel('epoch')
    title=str(i) + ' epoch'
    ax1.set_title(title, fontsize=24)

    plt.savefig('my_plot.png')
    if run_neptune:
        run = neptune.init_run(
            project=project_name,
            api_token=api_token,
            name=savename
        )
        run['parameters'] = {
            'learning_rate': lr,
            'channels': channels,
            'phys_loss_weight': phys_loss_weight,
            'n_epochs': n_epochs,
            'sigmoid_weight': sigmoid_weight,
            'clusters': clusters,
            'warmup_steps': warmup_steps,
            'field': point_cloud,
            'sources': sources,
            'sinks': sinks,
            'dt': dt,
            'boundary': boundary,
            'n_steps': n_steps,
            'seed':seed,
            'num_layers':num_layers,
            'temperature':temperature,
            'denseconv':denseconv

        }
        checkpoint = {
            'loss_curve': loss_curve,
            'adjacency': adjacency,
            'optimized_coords': out,
            'optimized_dmon': dmon.state_dict(),
        }
        run['plots/my_plot'].upload('my_plot.png')
        torch.save(checkpoint, 'checkpoint1.pth')
        run['checkpoints/checkpoint1.pth'].upload('checkpoint1.pth')
        run.stop()
        print("neptune work")
    plt.show()
    return

In [ ]:
torch.autograd.set_detect_anomaly(False)

##train function

In [ ]:

from matplotlib.patches import Polygon as Polygon_plt
from matplotlib.patches import Rectangle

def construct_adj_matrix(edge_index):
    adj = to_dense_adj(edge_index)
    deg = degree(edge_index[0])
    deg_sqrt_inv = deg.pow(-0.5)
    return deg_sqrt_inv.view(-1, 1) * adj * deg_sqrt_inv.view(1, -1)

def calc_h(coords, edge_index):
    x_i, x_j = coords[edge_index[0]], coords[edge_index[1]]
    return torch.norm(x_i - x_j, dim=1)

def calc_physics_weights(k, e, h, edge_index):
    k_i, k_j = k[edge_index[0]], k[edge_index[1]]
    numerator = 2 * k_i * k_j * e
    denominator = (k_i + k_j) * h + 1e-8
    assert numerator.shape == denominator.shape
    return torch.divide(numerator, denominator)



def train(point_cloud, boundary, sources, sinks, ground_truth_dynamics,
              dt=0.001, n_steps=10000, n_epochs=100, clusters=30, channels = [3,8], seed=42,
              lr = 0.03, warmup_steps = 50, phys_loss_weight = 50,sigmoid_weight=30,savename='test_run',
              run_wandb=False,run_neptune=False,api_token="", save_ani = False, denseconv = False,project_name="",num_layers=1,
              temperature=1.0,reg_loss_weight=0.0001,n_steps_val=200,n_steps_test=800,stop_bound=55,area_total=1.0, coarsener=None, optimizer=None,
              phys_loss_values = None, stability_loss_values = None, reg_loss_values = None, loss_curve = None, curr_epoch = 0,
              mse_val=-100.0, do_step=True, orig_mesh = None, orig_edge_index = None, orig_areas = None, orig_e = None, orig_clipped_vertices_dict = None, loss_fn = None, task_field_tensor = None,
              orig_h = None, orig_perm = None, orig_edge_weights = None,gatconv=True,orig_boundary_nodes = None):
    #start = time.process_time()
    #print(ground_truth_dynamics[:,:n_steps].shape[1], n_steps)
    assert ground_truth_dynamics[:,:n_steps].shape[1] == n_steps
    if task_field_tensor is None:
        #print("Created tensor")
        FIELD = torch.tensor(point_cloud, requires_grad=False, dtype=torch.float)
        #print(FIELD)
    else:
        print("Transferred tensor")
        FIELD = task_field_tensor.detach().contiguous()
        #print(FIELD)


    if loss_fn is None:
        loss_fn = MSELoss()
    if loss_curve is None:
      loss_curve = []

    u0 = torch.zeros((clusters + len(sources) + len(sinks), 1), requires_grad=False)

    p_bhp = torch.zeros((clusters + len(sources) + len(sinks), 1), requires_grad=False)
    w = torch.zeros((clusters + len(sources) + len(sinks), 1), requires_grad=False)

    for i, (_, power) in enumerate(sources.items()):
        p_bhp[clusters + len(sinks) + i] = power
        w[clusters + len(sinks) + i] = 1.

    keep = [i for i in range(clusters, clusters + len(sinks))]
    #print(keep)

    with torch.no_grad():
      if orig_mesh is None:
        #print("Time before initial differentiable_voronoi call: ", time.process_time() - start)
        mesh = triangulate(FIELD[:, :2].clone().detach().numpy())
        #print(FIELD.shape, boundary.shape)
        edge_index, areas, e, clipped_vertices_dict,boundary_nodes = differentiable_voronoi(FIELD[:, :2], mesh, boundary)
        #print("Time after initial differentiable_voronoi call: ", time.process_time() - start)
        if denseconv:
            adjacency = construct_adj_matrix(to_undirected(edge_index))
        orig_edge_index = edge_index
        perm = FIELD[:, 2]
        h = calc_h(FIELD[:, :2], edge_index)
        edge_weights = calc_physics_weights(perm, e, h, edge_index)
        edge_index, edge_weights = to_undirected(edge_index, edge_weights)


      else:
        mesh = orig_mesh
        edge_index = orig_edge_index
        areas = orig_areas
        e = orig_e
        clipped_vertices_dict = orig_clipped_vertices_dict
        if denseconv:
            adjacency = construct_adj_matrix(to_undirected(edge_index))
        h = orig_h
        perm = orig_perm
        edge_weights = orig_edge_weights
        edge_index, edge_weights = to_undirected(edge_index, edge_weights)
        boundary_nodes = orig_boundary_nodes

    # print('shape edge_weights ',edge_weights.shape)
    # print('shape edge_index ',edge_index.shape)
    # edge_index_print, weights_print = to_undirected(edge_index, edge_weights)
    # print('shape to_undirected(edge_weights) ',weights_print.shape)
    # print('shape to_undirected(edge_index) ',edge_index_print.shape)

    if coarsener is None:
      coarsener = learnableCoarsening(channels, clusters, sources=sources, sinks=sinks, denseconv=denseconv,num_layers=num_layers,temperature=temperature,seed=seed,gatconv=gatconv,boundary_nodes=list(boundary_nodes))
    else:
      coarsener.sources = torch.tensor(list(sources.keys()), dtype=torch.int)
      coarsener.sinks = torch.tensor(sinks, dtype=torch.int)
      coarsener.boundary_nodes = torch.tensor(list(boundary_nodes),dtype=torch.int)
    if optimizer is None:
      optimizer = torch.optim.Adam(coarsener.parameters(), lr=lr)

    solver = Solver()
    stability_criterion = StabilityCriterion(n = clusters,s=area_total)
    stability_criterion_check = StabilityCriterionCheck()
    physical_loss = None
    reduced_dynamics = None
    if phys_loss_values is None:
      phys_loss_values = []
    if stability_loss_values is None:
      stability_loss_values = []
    if reg_loss_values is None:
      reg_loss_values=[]

    phys_start=warmup_steps

    current_mse_val=0.0

    qounter_val_stop=0
    qounter_bound=stop_bound

    #print(weights.shape)

    #print(time.process_time() - start)
    for i in range(curr_epoch, curr_epoch + n_epochs):
        if do_step:
          optimizer.zero_grad()

        unsqueezed_field = torch.unsqueeze(FIELD, 0)
        print(unsqueezed_field.grad)

        if denseconv:
          s, out, bboxes = coarsener(unsqueezed_field, adjacency, edge_weights)
        else:
          print('shape edge_weights ',edge_weights.shape)
          print('shape edge_index ',edge_index.shape)
          s, out, bboxes = coarsener(unsqueezed_field, edge_index, edge_weights)
        #print(time.process_time() - start)

        with torch.no_grad():
            #print(out[0][:, :2].detach().numpy().max(), out[0][:, :2].detach().numpy().min())

            mesh = triangulate(out[0][:, :2].clone().detach().numpy())

        try:
            coarsened_edge_index, coarsened_areas, coarsened_e, coarsened_clipped_vertices_dict,coarsened_boundary_nodes = differentiable_voronoi(out[0][:, :2], mesh, boundary)
        except:
            print(out[0])
        coarsened_h = calc_h(out[0][:, :2], coarsened_edge_index)

        edge_index_2, [e_2, h_2] = to_undirected(coarsened_edge_index, [coarsened_e, coarsened_h])
        stability_loss = sigmoid_weight*torch.sigmoid(stability_criterion(v=coarsened_areas, k=out[0][:, 2],
                                                              e=e_2.view(-1, 1), h=h_2.view(-1, 1),
                                                              edge_index=edge_index_2))
        # stability_loss = stability_criterion(v=areas, k=out[0][:, 2],
        #                                                       e=e_2.view(-1, 1), h=h_2.view(-1, 1),
        #                                                       edge_index=edge_index_2)
        with torch.no_grad():
          max_stable_time = stability_criterion_check(v=coarsened_areas, k=out[0][:, 2],
                                                              e=e_2.view(-1, 1), h=h_2.view(-1, 1),
                                                              edge_index=edge_index_2)
          #print("Current max stable time: ", max_stable_time)

        if i >= warmup_steps: # after warm-up period we turn on the physical loss, provided that the stability check of the numerical scheme is passed

            if max_stable_time > 2*dt: # the condition for the stability of the circuit is fulfilled

              reduced_dynamics, _ = roll_out(solver, n_steps=n_steps+n_steps_val, u0=u0, v=coarsened_areas, h=coarsened_h, e=coarsened_e,
                                            k=out[0][:,2], w=w, p_bhp=p_bhp, dt=dt,
                                            edge_index=coarsened_edge_index, keep=keep)
              #print("reduced_dynamics shape: ", reduced_dynamics.shape)
              assert ground_truth_dynamics.shape == reduced_dynamics.shape
              #print(time.process_time() - start)
              #physical_loss = torch.mean(torch.sqrt(torch.mean((ground_truth_dynamics[:,:n_steps] - reduced_dynamics[:,:n_steps])**2, dim=1)))/(torch.sqrt(torch.mean(ground_truth_dynamics[:, :n_steps] ** 2)) + 1e-8)*phys_loss_weight
              '''
                physical_loss = torch.mean(
    torch.mean((ground_truth_dynamics[:, :n_steps] - reduced_dynamics[:, :n_steps]) ** 2, dim=1)
    / (torch.mean(ground_truth_dynamics[:, :n_steps] ** 2, dim=1) + 1e-8)
) * phys_loss_weight
            '''
              #physical_loss = torch.mean(torch.sqrt(torch.mean((ground_truth_dynamics[:,:n_steps] - reduced_dynamics[:,:n_steps])**2, dim=1))/(torch.sqrt(torch.mean(ground_truth_dynamics[:, :n_steps] ** 2, dim=1)) + 1e-8))*phys_loss_weight
              physical_loss = torch.mean(torch.sqrt(torch.mean((ground_truth_dynamics[:,:n_steps] - reduced_dynamics[:,:n_steps])**2, dim=1)))*phys_loss_weight
              if n_steps_val>0:
                  if mse_val<0:
                      mse_val=torch.mean(torch.sqrt(torch.mean((ground_truth_dynamics[:,n_steps:n_steps+n_steps_val] - reduced_dynamics[:,n_steps:n_steps+n_steps_val])**2, dim=1)))

                  current_mse_val=torch.mean(torch.sqrt(torch.mean((ground_truth_dynamics[:,n_steps:n_steps+n_steps_val] - reduced_dynamics[:,n_steps:n_steps+n_steps_val])**2, dim=1)))
                  if mse_val>current_mse_val:
                      mse_val=current_mse_val
                      qounter_val_stop=0
                  else:
                      qounter_val_stop+=1

              #print(time.process_time() - start)


              phys_loss_values.append(physical_loss.item())


              loss = stability_loss[0] + physical_loss
              '''
              for name, param in coarsener.named_parameters():
                print("Gradients before backward()")
                print(name, param.grad)
              '''

              loss.backward()
              '''
              for name, param in coarsener.named_parameters():
                print("Gradients after backward()")
                print(name, param.grad)
              '''
            else: # We adjust the loss for stability until the current dt becomes stable
              loss = stability_loss[0]
              if len(phys_loss_values) > 0:
                phys_loss_values.append(phys_loss_values[len(phys_loss_values) - 1]) # we take it from the previous step
              loss.backward()
            if len(phys_loss_values)==1:
                phys_start=i
        else:

          loss = stability_loss[0]
          loss.backward()
        #print(stability_loss[0])
        stability_loss_values.append(stability_loss[0].item())

        loss_curve.append(loss.item())
        #print(time.process_time() - start)

        # Apply gradient clipping
        if do_step:
          nn_utils.clip_grad_norm_(coarsener.parameters(), max_norm=0.2)

          optimizer.step()

        if qounter_val_stop>qounter_bound:
            break

        with torch.no_grad():

            if  i==(n_epochs-1) and run_neptune:
                print_with_neptune(out,coarsener, clipped_vertices_dict, point_cloud, boundary,ground_truth_dynamics, reduced_dynamics, adjacency,
                            loss_curve,phys_loss_values,stability_loss_values,s, sources=sources, sinks=sinks,run_neptune=run_neptune,
                            dt=dt, n_steps=n_steps, n_epochs=n_epochs, clusters=clusters, channels = channels, lr = lr,
                            warmup_steps =warmup_steps, phys_loss_weight = phys_loss_weight,sigmoid_weight=sigmoid_weight,
                            savename=savename,seed=seed,unit='$K(x, y)$',i=i,api_token=api_token,phys_start=phys_start,project_name=project_name,num_layers=num_layers,temperature=temperature,denseconv=denseconv)





            if i > 1:

                if i % 20 == 0 or i==(n_epochs-1):

                  clear_output(wait=True)

                  sources_reduced= list(range(len(out[0]) - len(sources),  len(out[0])))
                  sinks_reduced= list(range(len(out[0]) - len(sources) - len(sinks),  len(out[0]) - len(sources)))
                  fig = plt.figure(figsize=(30, 10))
                  gs = fig.add_gridspec(2,2)
                  ax1 = fig.add_subplot(gs[:, 0])
                  ax2 = fig.add_subplot(gs[0, 1])
                  ax3 = fig.add_subplot(gs[1, 1])



                  for vertex in coarsened_clipped_vertices_dict.keys():
                      line = coarsened_clipped_vertices_dict[vertex].clone().detach()
                      cmap = plt.get_cmap('RdYlBu')
                      p = matplotlib.patches.Polygon(line, facecolor = 'g', edgecolor='black', alpha=0.2, fill=False)
                      m = torch.mean(torch.unique(line, dim=0), dim=0)
                      ax1.add_patch(p)
                  scale = ax1.scatter(out[0].detach()[:, 0], out[0].detach()[:, 1], c=out[0].detach().numpy()[:, 2], s=400)
                  if sources is not None:
                      for j, src in enumerate(sources_reduced):
                          ax1.scatter(out[0].detach()[src, 0].item(), out[0].detach()[src, 1].item(), s=100, c='r', marker='v')
                          ax1.text(out[0].detach()[src, 0].item(), out[0].detach()[src, 1].item(), 'source ' + str(j+1), fontsize=30)

                  if sinks is not None:
                      for j, snk in enumerate(sinks_reduced):
                          ax1.scatter(out[0].detach()[snk, 0].item(), out[0].detach()[snk, 1].item(), s=100, c='g', marker='^')
                          ax1.text(out[0].detach()[snk, 0].item(), out[0].detach()[snk, 1].item(), 'sensor ' + str(j+1), fontsize=30)
                  '''
                  k = len(windows)
                  cmap = plt.get_cmap('tab20', k)  # любая качественная карта c k цветами

                  for idx, poly in enumerate(windows):
                      pts = poly.cpu().numpy()  # из Tensor в numpy
                      color = cmap(idx)
                      patch = Polygon_plt(
                          pts,
                          closed=True,
                          edgecolor=color,
                          facecolor='none',
                          linestyle='--',    # пунктир
                          linewidth=2,
                          alpha=0.8
                      )
                      ax1.add_patch(patch)
                  '''

                  cbar = plt.colorbar(scale)
                  cbar.ax.tick_params(labelsize=24)
                  cbar.ax.set_ylabel('$K(x, y)$', fontsize=40)

                  if reduced_dynamics is not None:
                      for pp in range(len(sinks_reduced)):
                          ax2.plot(reduced_dynamics[pp,:n_steps], label='reduced_sensor '+str(pp+1))
                          ax2.plot(ground_truth_dynamics[pp,:n_steps], label='gt_sensor '+str(pp+1))
                  #ax2.set_title('$u(t)$ at sinks')
                  ax2.set_xlabel('time step')
                  ax2.set_ylabel('$p_{s},~Pa$')
                  ax2.legend()

                  label_stability = 'stability loss'
                  ax3.plot(stability_loss_values, label=label_stability)
                  if phys_loss_values is not None:
                      label_phys = 'phys loss'
                      ax3.plot(list(range(phys_start, phys_start + len(phys_loss_values))), phys_loss_values, label=label_phys)

                  ax3.legend()
                  ax3.set_yscale('log')
                  ax3.set_title('Loss')
                  ax3.set_xlabel('epoch')
                  ax3.set_ylabel('Loss')
                  title=str(i) + ' epoch'
                  ax1.set_title(title, fontsize=24)

                  if save_ani:
                    plt.savefig(f'/content/ani/{str(i).zfill(10)}')
                  plt.show()

                  #sleep(4)

                  #clear_output(wait=True)





    return loss_curve, coarsener, 0, out.detach()

In [ ]:
from datetime import datetime

def train_competing(point_cloud, boundary, sources, sinks, ground_truth_dynamics, clamp_x = [0, 1.0], clamp_y = [0, 1.0], dt=0.001, n_steps=10000, n_epochs=100,save_ani=False):
    assert ground_truth_dynamics.shape[1] == n_steps
    COORDS = torch.tensor(point_cloud[:, :2], requires_grad=True)
    optimizer = torch.optim.Adam([COORDS], lr=0.002)
    loss_fn = MSELoss()
    loss_curve = []
    mesh = triangulate(COORDS.clone().detach())

    mask = torch.zeros((len(COORDS), 1))
    u0 = torch.zeros((len(COORDS), 1), requires_grad=False)
    perm = torch.tensor(point_cloud[:, 2], requires_grad=False)

    p_bhp = torch.zeros((len(COORDS), 1), requires_grad=False)
    w = torch.zeros((len(COORDS), 1), requires_grad=False)
    for s, power in sources.items():
        p_bhp[s] = power
        w[s] = 1
        mask[s] = 1
    for s in sinks:
        mask[s] = 1

    solver = Solver()

    stability_criterion_check = StabilityCriterionCheck()

    for i in range(1, n_epochs):
        optimizer.zero_grad()
        C_n = COORDS.clone().detach()

        edge_index, areas, e, clipped_vertices_dict,_ = differentiable_voronoi(COORDS, mesh, boundary)
        h = calc_h(COORDS, edge_index)
        edge_index_2, [e_2, h_2] = to_undirected(edge_index, [e, h])
        with torch.no_grad():
          max_stable_time = stability_criterion_check(v=areas, k=point_cloud[:, 2],
                                                                e=e_2.view(-1, 1), h=h_2.view(-1, 1),
                                                                edge_index=edge_index_2)
          #print("Current max stable time: ", max_stable_time)
        reduced_dynamics, _ = roll_out(solver, n_steps=n_steps, u0=u0, v=areas, h=h, e=e, k=perm, w=w, p_bhp=p_bhp, dt=dt,
                                    edge_index=edge_index, keep=sinks)

        loss = torch.mean(torch.sqrt(torch.mean((ground_truth_dynamics - reduced_dynamics)**2, dim=1)))
        loss_curve.append(loss.item())
        #print(i, loss.item())
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            COORDS[:] = COORDS - mask * (COORDS - C_n)
            COORDS[:, 0] = COORDS[:, 0].clamp(clamp_x[0], clamp_x[1])
            COORDS[:, 1] = COORDS[:, 1].clamp(clamp_y[0], clamp_y[1])
            mesh = triangulate(COORDS.clone().detach())

            if i % 5 == 0:
                clear_output(wait=True)
                #print(i, loss.item())
                print("Before printing: ", COORDS[:,0].max(), COORDS[:,1].min())
                ICML_print(COORDS, clipped_vertices_dict, perm, ground_truth_dynamics, reduced_dynamics, loss_curve,
                           sources=sources, sinks=sinks, title=str(i) + ' epoch', file_title = datetime.now().strftime('%Y-%m-%d_%H-%M-%S'), i=i,save_ani=save_ani)

    result_field = torch.cat((COORDS, torch.from_numpy(point_cloud)[:, 2].unsqueeze(1)), dim=1)

    return loss_curve, result_field

In [ ]:
def reduce_graph(graph, k, sources=[0], sinks=[1]):
    labels = KMeans(k, random_state=17).fit_predict(graph[:, :2])
    loop_reduced = avg_pool_x(torch.tensor(labels), torch.tensor(graph), torch.ones(len(labels)))[0]
    loop_reduced = torch.cat([loop_reduced, torch.tensor(graph[list(sources.keys())]), torch.tensor(graph[sinks])], dim=0)
    loop_reduced, return_inverse, _ = torch.unique(loop_reduced, return_inverse=True, return_counts=True,  dim=0)
    loop_reduced_saved = loop_reduced

    for i, p in enumerate(loop_reduced):
        for j, n in enumerate(loop_reduced):
            if i != j and torch.norm(p - n) < 1e-4:
                raise ValueError

    return loop_reduced.detach().numpy(), return_inverse[-len(sources) - len(sinks): - len(sinks)], return_inverse[-len(sinks):]

In [ ]:
def try_different_degrees_of_reduction(orig_field, boundary, dt, n_steps, reduction_degrees=[0.75, 0.5, 0.25],
                                                    n_epochs=100, aggregating_function=KMeans, method='gnn',
                                                    sources = None, sinks = None,clusters=30, channels = [3,8], seed=42,
                                                    lr = 0.03, warmup_steps = 50, phys_loss_weight = 50,sigmoid_weight=30
                                                    ,savename='test_run',run_wandb=False,run_neptune=False,clamp_x = [0, 10.0], clamp_y = [0, 10.0], api_token="",
                                                    source_power = 100.0, denseconv=False, save_ani=False,temperature=1.0,num_layers=1,project_name="",n_steps_val=0,stop_bound=55,top_k=1.0,gatconv = True):
    ground_truth_dynamics, ani = dynamics(orig_field, sources=sources,
                             sinks=sinks, boundary=boundary,
                             dt=dt, save_animation=True, n_steps=n_steps+n_steps_val)
    print(len(ground_truth_dynamics[:,:n_steps]))
    N = len(orig_field)

    storage = {}
    storage[1] = ground_truth_dynamics[:,:n_steps]
    coarseners = {}
    for r in reduction_degrees:
        n = int(r * N)
        assert n > 0
        if method == 'points' or method == 'kmeans':
          reduced_point_cloud, sources_reduced, sinks_reduced = reduce_graph(orig_field, n, sources = sources, sinks = sinks) # k-means (initial)
          print("sources_reduced",sources_reduced,"sinks_reduced",sinks_reduced)
          sources_reduced = {src: source_power for src in sources_reduced}

        if method != 'kmeans':
          if method == 'points':
            loss_curve, reduced_point_cloud = train_competing(reduced_point_cloud, boundary,
                                    {src: source_power for src in sources_reduced},sinks_reduced, ground_truth_dynamics[:,:n_steps],
                                    clamp_x = clamp_x, clamp_y = clamp_y, dt=dt, n_steps=n_steps,
                                    n_epochs=n_epochs,save_ani=save_ani)
          elif method == 'gnn':
            loss_curve, coarsener, adjacency, reduced_point_cloud = train(orig_field,
                  sources=sources,
                  sinks=sinks,
                  ground_truth_dynamics=ground_truth_dynamics,
                  boundary=boundary,
                  dt=dt, n_steps=n_steps, n_epochs=n_epochs,clusters=n, channels = channels, seed=seed,
                  lr = lr, warmup_steps = warmup_steps, phys_loss_weight = phys_loss_weight,
                  sigmoid_weight=sigmoid_weight,savename=savename,run_wandb=run_wandb, run_neptune=run_neptune,api_token=api_token,project_name=project_name,
                  denseconv=denseconv, save_ani=save_ani,temperature=temperature,num_layers=num_layers,n_steps_val=n_steps_val,stop_bound=stop_bound,gatconv=gatconv)
            reduced_point_cloud = reduced_point_cloud[0]

            sources_reduced = list(range(len(reduced_point_cloud) - len(sources),  len(reduced_point_cloud)))
            sources_reduced = {src: source_power for src in sources_reduced}
            sinks_reduced = list(range(len(reduced_point_cloud) - len(sources) - len(sinks),  len(reduced_point_cloud) - len(sources)))
            print(sources_reduced, sinks_reduced)
            coarseners[r]=coarsener

        reduced_dynamics, _ = dynamics(reduced_point_cloud, sources_reduced,sinks_reduced, boundary, n_steps=3*n_steps, dt = dt)

        storage[r] = reduced_dynamics, reduced_point_cloud

    if method == "gnn":
      return storage, coarseners
    else:
      return storage

## Sinusoidal variable permeability scenario

In [ ]:
l = 1
x, y = np.meshgrid(np.linspace(0, l, 20), np.linspace(0, l, 20))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)
k = np.cos(20 * x)+ np.sin(y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task1 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [210]},
        'sinks': [42, 399-42],
        'dt': 0.00001,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 1000}

In [ ]:
scatter = plt.scatter(task1['field'][:, 0], task1['field'][:, 1], c=task1['field'][:, 2])
#plt.title("Original permeability field (cos(20x) + sin(y))")
# s = list(task1['sources'].keys())
# plt.scatter(x[s], y[s], marker='D', c='m')
ii=0
for j in task1['sinks']:
  plt.scatter(task1['field'][j,0], task1['field'][j,1], s=100, c='g', marker='^')
  plt.annotate('sensor'+str(ii+1), (task1['field'][j,0], task1['field'][j,1]),xytext=(-20.0, 0),textcoords='offset points', fontsize=16)
  ii+=1

ii=0
for j in task1['sources'].keys():
  plt.scatter(task1['field'][j,0], task1['field'][j,1], s=100, c='r', marker='v')
  plt.annotate('source'+str(ii+1), (task1['field'][j,0], task1['field'][j,1]), fontsize=16)
  ii+=1
plt.colorbar(scatter)

In [ ]:
ground_truth, ani = dynamics(task1['field'], sources=task1['sources'],
                             sinks=task1['sinks'], boundary=task1['boundary'],
                             dt=task1['dt'], save_animation=False, n_steps=task1['n_steps'])

In [ ]:
set_seeds(42)
storage_01, coarsener_01 = try_different_degrees_of_reduction(task1['field'], task1['boundary'], task1['dt'], task1['n_steps'],
                                                        reduction_degrees=[0.1], n_epochs=300, lr = 0.02, sigmoid_weight = 0.1, phys_loss_weight = 100.0,
                                                        aggregating_function=KMeans, method='gnn', sources = task1['sources'], sinks = task1['sinks'], save_ani=False,
                                                        denseconv=False, num_layers=3,temperature=0.4,n_steps_val=0,warmup_steps=50,gatconv = True)


In [ ]:
set_seeds(42)
storage_0075, coarsener_0075 = try_different_degrees_of_reduction(task1['field'], task1['boundary'], task1['dt'], task1['n_steps'],
                                                        reduction_degrees=[0.075], n_epochs=300, lr = 0.008, sigmoid_weight = 0.1, phys_loss_weight = 100.0,
                                                        aggregating_function=KMeans, method='gnn', sources = task1['sources'], sinks = task1['sinks'], save_ani=False,
                                                        denseconv=False, num_layers=3,temperature=0.4,n_steps_val=0,warmup_steps=50,gatconv = True)


In [ ]:
set_seeds(42)
storage_005, coarsener_005 = try_different_degrees_of_reduction(task1['field'], task1['boundary'], task1['dt'], task1['n_steps'],
                                                        reduction_degrees=[0.05], n_epochs=300, lr = 0.01, sigmoid_weight = 0.1, phys_loss_weight = 100.0,
                                                        aggregating_function=KMeans, method='gnn', sources = task1['sources'], sinks = task1['sinks'], save_ani=False,
                                                        denseconv=False, num_layers=3,temperature=0.4,n_steps_val=0,warmup_steps=50,gatconv = True)


In [ ]:
path='/content/gdrive/MyDrive/ML_ST/new_storage_sin_01.pt'
torch.save(storage_01, path)

path='/content/gdrive/MyDrive/ML_ST/new_storage_sin_0075.pt'
torch.save(storage_0075, path)

path='/content/gdrive/MyDrive/ML_ST/new_storage_sin_005.pt'
torch.save(storage_005, path)

In [ ]:
path='/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_01.pt'
torch.save(coarsener_01, path)

path='/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_0075.pt'
torch.save(coarsener_0075, path)

path='/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_005.pt'
torch.save(coarsener_005, path)

In [ ]:
storage_01 = torch.load('/content/gdrive/MyDrive/ML_ST/new_storage_sin_01.pt')

##Metis

In [ ]:
def metis_coarsening_sin(path,n_clusters):
    data = []
    with open(path, "r") as file:
        for line in file:
            row = [float(x) for x in line.split()]
            row[-1] = int(row[-1])
            data.append(row)

    data = np.array(data)

    data[42][3]=n_clusters+1
    data[399-42][3]=n_clusters+2
    data[210][3]=n_clusters+3

    clusters = np.unique(data[:, 3])
    centers_of_mass = {}




    for cluster in clusters:
        points = data[data[:, 3] == cluster][:, :3]
        center=[0,0,0]
        points[:,0]=points[:,0]*points[:,2]
        points[:,1]=points[:,1]*points[:,2]
        center[0]=points[:,0].sum()/points[:,2].sum()
        center[1]=points[:,1].sum()/points[:,2].sum()
        center[2]=points[:,2].mean()
        centers_of_mass[cluster] = center

    centers_xy=[]
    for cluster_id, center in centers_of_mass.items():
        centers_xy.append(center)

    #centers_xy.append(field[180])
    #centers_xy.append(field[0])

    centers_xy=np.array(centers_xy)

    x = centers_xy[:,0].reshape(-1, 1)
    y = centers_xy[:,1].reshape(-1, 1)
    k = centers_xy[:,2].reshape(-1, 1)
    eps = 1e-1
    task2 = {'field': np.concatenate([x, y, k], axis=1),
            'sources': {src: 100. for src in [n_clusters+2]},
            'sinks': [n_clusters,n_clusters+1],
            'dt': 0.00001,
            'boundary': torch.tensor([[-eps,-eps],
                                      [-eps, l + eps],
                                      [l + eps, l + eps],
                                        [l + eps, -eps]], requires_grad=False),
            'n_steps': 1000}

    return task2




In [ ]:
metis_sin_01_field=metis_coarsening_sin('/content/output_points_sin_40.txt',40)
metis_sin_0075_field=metis_coarsening_sin('/content/output_points_sin_30.txt',30)
metis_sin_005_field=metis_coarsening_sin('/content/output_points_sin_20.txt',20)

metis_dynamics_01_sin, ani = dynamics(metis_sin_01_field['field'], sources=metis_sin_01_field['sources'],
                             sinks=metis_sin_01_field['sinks'], boundary=task1['boundary'],
                             dt=task1['dt'], save_animation=False, n_steps=3*task1['n_steps'])
metis_dynamics_0075_sin, ani = dynamics(metis_sin_0075_field['field'], sources=metis_sin_0075_field['sources'],
                             sinks=metis_sin_0075_field['sinks'], boundary=metis_sin_0075_field['boundary'],
                             dt=metis_sin_0075_field['dt'], save_animation=False, n_steps=3*metis_sin_0075_field['n_steps'])
metis_dynamics_005_sin, ani = dynamics(metis_sin_005_field['field'], sources=metis_sin_005_field['sources'],
                             sinks=metis_sin_005_field['sinks'], boundary=task1['boundary'],
                             dt=task1['dt'], save_animation=False, n_steps=3*task1['n_steps'])

##Load sinusoidal

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# path='/content/gdrive/MyDrive/icml_2025/new_storage_sin_01_val_200.pt'
# storage_01 = torch.load(path)

# path='/content/gdrive/MyDrive/icml_2025/new_storage_sin_005_val_500.pt'
# storage_005 = torch.load(path)

# path='/content/gdrive/MyDrive/icml_2025/new_storage_sin_0075_val_200.pt'
# storage_0075 = torch.load(path)

In [ ]:
result_competing_method=torch.load('/content/gdrive/MyDrive/icml_2025/result_competing_method_sinusoidal_lr_0.005.pt')
#result_competing_method=torch.load('/content/gdrive/MyDrive/result_competing_method_sinusoidal_lr_0.005.pt')


In [ ]:
#result_competing_method[0.05]=torch.load('/content/result_old_method_0.05_sinks_2_sources_1.pt')

In [ ]:
x = torch.load('/content/result_old_method_0.05_sinks_2_sources_1.pt')

In [ ]:
storage_01 = torch.load('/content/gdrive/MyDrive/ML_ST/new_storage_sin_01.pt')
storage_0075 = torch.load('/content/gdrive/MyDrive/ML_ST/new_storage_sin_0075.pt')
storage_005 = torch.load('/content/gdrive/MyDrive/ML_ST/new_storage_sin_005.pt')

##Printing

In [ ]:
def ICLR_compare_rmse_2(stores, labels,start,end):
    fig, ax = plt.subplots(figsize=(14, 10))
    markers = iter(["o", "s", "D"])
    for i, res in enumerate(stores):
        errors = {}
        gt = res[1][start:end]
        for k, v in res.items():
            if k == 1: continue
            error = np.log10(np.sqrt(mean_squared_error(gt, v[start:end])))
            errors[k] = error
        errors = dict(sorted(errors.items()))
        ax.scatter(list(errors.keys()), list(errors.values()), label=labels[i], marker=next(markers), s=200)
    plt.legend(fontsize=20, loc='upper right')
    ax.set_ylabel('RMSE, log10 scale', fontsize=20)
    ax.set_xlabel('r', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.grid(True)
    plt.show()

In [ ]:
def ICLR_compare_curves_2(result,end):
    fig, ax = plt.subplots(figsize=(14, 10))
    for k, v in result.items():
        ax.plot(v[:end], label='r=' + str(round(k, 3)), linewidth=4)
    plt.legend(fontsize=20)
    ax.set_ylabel('$p_{s},~Pa$', fontsize=20)
    ax.set_xlabel('time steps', fontsize=20)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.grid(True)
    plt.show()

In [ ]:
def ICLR_compare_rmse_2(stores, labels,start,end):
    fig, ax = plt.subplots(figsize=(14, 10))
    markers = iter(["o", "s", "D","^"])
    for i, res in enumerate(stores):
        errors = {}
        gt = res[1][start:end]
        for k, v in res.items():
            if k == 1: continue
            error = np.log10(np.sqrt(mean_squared_error(gt, v[start:end])))
            errors[k] = error
        errors = dict(sorted(errors.items()))
        ax.scatter(list(errors.keys()), list(errors.values()), label=labels[i], marker=next(markers), s=200)
        #ax.scatter(list(errors.keys()), list(errors.values()), label=labels[i], s=200)
    plt.legend(fontsize=20, loc='upper right')
    ax.set_ylabel('RMSE, log10 scale', fontsize=20)
    ax.set_xlabel('r', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.grid(True)
    plt.show()

In [ ]:
result_kmeans = try_different_degrees_of_reduction(task1['field'], task1['boundary'], task1['dt'], task1['n_steps'], reduction_degrees=[0.1, 0.075, 0.05], n_epochs=300,
                                                   lr = 0.1, aggregating_function=KMeans, method='kmeans', sources = task1['sources'], sinks = task1['sinks'])

In [ ]:
l = 1
x, y = np.meshgrid(np.linspace(0, l, 20), np.linspace(0, l, 20))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)
k = np.cos(20 * x)+ np.sin(y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task1 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [210]},
        'sinks': [42, 399-42],
        'dt': 0.00001,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 1000}

ground_truth_2000, ani = dynamics(task1['field'], sources=task1['sources'],
                             sinks=task1['sinks'], boundary=task1['boundary'],
                             dt=task1['dt'], save_animation=False, n_steps=3*task1['n_steps'])



In [ ]:
orig_curve = ground_truth_2000

result_new_01 = storage_01[0.1]
result_new_005 = storage_005[0.05]
result_new_0075 = storage_0075[0.075]

result_competing_01 = result_competing_method[0.1]
result_competing_005 = result_competing_method[0.05]
result_competing_0075 =  result_competing_method[0.075]

result_kmeans_01 = result_kmeans[0.1]
result_kmeans_005 =result_kmeans[0.05]
result_kmeans_0075 = result_kmeans[0.075]

result_new = {}
result_new[1] = orig_curve[0]
result_new[0.1] = result_new_01[0][0].tolist()
result_new[0.05] = result_new_005[0][0].tolist()
result_new[0.075] = result_new_0075[0][0].tolist()
result_competing = {}
result_competing[1] = orig_curve[0]
result_competing[0.1] = result_competing_01[0][0].tolist()
result_competing[0.05] = result_competing_005[0][0].tolist()
result_competing[0.075] = result_competing_0075[0][0].tolist()

In [ ]:
# result_new = {}
# result_new[1] = orig_curve[0]
# result_new[0.1] = result_new_01[0][0].tolist()
# result_new[0.05] = result_new_005[0][0].tolist()
# result_new[0.075] = result_new_0075[0][0].tolist()
# result_competing = {}
# result_competing[1] = orig_curve[0]
# result_competing[0.1] = result_competing_01[0][0].tolist()
# result_competing[0.05] = result_competing_005[0][0].tolist()
# result_competing[0.075] = result_competing_0075[0][0].tolist()

result_kmeans_ = {}
result_kmeans_[1] = orig_curve[0]
result_kmeans_[0.1] = result_kmeans_01[0][0].tolist()
result_kmeans_[0.05] = result_kmeans_005[0][0].tolist()
result_kmeans_[0.075] = result_kmeans_0075[0][0].tolist()

result_metis={}
result_metis[1] = orig_curve[0]
result_metis[0.1] = metis_dynamics_01_sin[0]
result_metis[0.05] = metis_dynamics_005_sin[0]
result_metis[0.075] = metis_dynamics_0075_sin[0]

In [ ]:
n_steps=1000
ICLR_compare_curves_2(result_new,n_steps)
ICLR_compare_curves_2(result_competing,n_steps)
ICLR_compare_curves_2(result_kmeans_,n_steps)
ICLR_compare_curves_2(result_metis,n_steps)

In [ ]:
ICLR_compare_rmse_2([result_competing, result_new, result_kmeans_,result_metis], ['Competing algorithm', 'Our algorithm', 'K-Means + AVG','Metis reduction'],0,1000)

In [ ]:
ICLR_compare_rmse_2([result_competing, result_new, result_kmeans_,result_metis], ['Competing algorithm', 'Our algorithm', 'K-Means + AVG','Metis reduction'],0,1000)

In [ ]:
len(result_new[0.1])

In [ ]:
result_new = {}
result_new[1] = orig_curve[1]
result_new[0.1] = result_new_01[0][1].tolist()
result_new[0.05] = result_new_005[0][1].tolist()
result_new[0.075] = result_new_0075[0][1].tolist()
result_competing = {}
result_competing[1] = orig_curve[1]
result_competing[0.1] = result_competing_01[0][1].tolist()
result_competing[0.05] = result_competing_005[0][1].tolist()
result_competing[0.075] = result_competing_0075[0][1].tolist()

result_kmeans = {}
result_kmeans[1] = orig_curve[1]
result_kmeans[0.1] = result_kmeans_01[0][1].tolist()
result_kmeans[0.05] = result_kmeans_005[0][1].tolist()
result_kmeans[0.075] = result_kmeans_0075[0][1].tolist()

result_metis={}
result_metis[1] = orig_curve[1]
result_metis[0.1] = metis_dynamics_01_sin[1]
result_metis[0.05] = metis_dynamics_005_sin[1]
result_metis[0.075] = metis_dynamics_0075_sin[1]

In [ ]:
n_steps=1000
ICLR_compare_curves_2(result_new,n_steps)
ICLR_compare_curves_2(result_competing,n_steps)
ICLR_compare_curves_2(result_kmeans,n_steps)
ICLR_compare_curves_2(result_metis,n_steps)

In [ ]:
ICLR_compare_rmse_2([result_competing, result_new, result_kmeans,result_metis], ['Competing algorithm', 'Our algorithm', 'K-Means + AVG','Metis reduction'],0,1000)

In [ ]:
ICLR_compare_rmse_2([result_competing, result_new, result_kmeans,result_metis], ['Competing algorithm', 'Our algorithm', 'K-Means + AVG','Metis reduction'],0,1000)

In [ ]:
ICLR_compare_rmse_2([result_competing, result_new, result_kmeans,result_metis], ['Competing algorithm', 'Our algorithm', 'K-Means + AVG','Metis reduction'],1000,2000)

##DMD sin

In [ ]:
def get_dmd_estimation(X_train, rank):
  est = dmd.DMD(svd_rank = rank, opt = True)
  t0 = time.perf_counter()
  #print('X_train.shape',X_train.shape)
  X_train = X_train.numpy()
  est.fit(X_train)
  t1 = time.perf_counter()
  est.dmd_time["tend"] *= 2
  return est.reconstructed_data, t1-t0


In [ ]:
def ICLR_compare_curves_different_reduction_algs(result,labels,title='',steps_split=1000):
    fig, ax = plt.subplots(figsize=(14, 10))
    for k, v in result.items():
        ax.plot(v, label=labels[k], linewidth=4)

    split_point = steps_split
    ax.axvline(x=split_point, color='r', linestyle='--')
    y_min, y_max = ax.get_ylim()
    ax.text(split_point, y_max*0.9, 'Train/Test Split', rotation=0, verticalalignment='center', color='black',fontsize=24)

    plt.legend(fontsize=20)
    ax.set_ylabel('$p_{s},~Pa$', fontsize=20)
    ax.set_xlabel('time steps', fontsize=20)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.title(title,fontsize = 20)
    plt.grid(True)
    plt.show()

result={}
result[0]=x[task1['sinks'][0],:2000]
result[1]=ground_truth_1[task1['sinks'][0],:2000]
result[2]=storage_01[0.1][0][0].tolist()[:2000]
ICLR_compare_curves_different_reduction_algs(result,["DMD","ground truth","our algorithm"],"sinusoidal permeability field")

result={}
result[0]=x[task1['sinks'][1]]
result[1]=ground_truth_1[task1['sinks'][1]]
result[2]=storage_01[0.1][0][1].tolist()[:2000]
ICLR_compare_curves_different_reduction_algs(result,["DMD","ground truth","our algorithm"],"sinusoidal permeability field")

##loop scenario

In [ ]:
image = '██░░░░░░░░░░░░░░░░░░░░░░███░░░░░░░░░░░░░░░░░░░░░░███░░░░░░░▄▄▄▄▄░░░░░░░░░░███░░░░█████████▄░░░░░░░░███▄░▐██▀░░░▀███░░░░░░░░░▀█████░░░░░░░██░░░░░░░░░░░█████▄▄▄▄▄███░░░░░░░░░░░░░█████████▀░░░░░░░░░░░░░░░███░░░░░░░░░░░░░░░░░░░░░░█████▄▄▄░░░░░░░░░░░░░░░░░░▀▀███████▄░░░░░░░░░░░░░░░░░░░▀▀████░░░░░░░░░░░░░░░░░░░░░░███'
def image_to_graph(image, image_height=13):
    image_width = int(len(image)/ image_height)
    xs = np.linspace(0, 1, image_width)
    ys = np.linspace(0, 1, image_height)
    xs, ys = np.meshgrid(xs, ys)
    xs, ys = xs.flatten(), ys.flatten()
    ks = []
    for i in image:
        if i == '░': ks.append(0.01)
        else: ks.append(1)
    return np.array([xs, ys, ks]).T
def plot_diagram(points, ax, title=''):
    vor = Voronoi(points[:, :2])
    voronoi_plot_2d(vor, ax=ax, show_vertices=False)
    ax.set_title(title)
    bar1 = ax.scatter(points[:, 0], points[:, 1], c=points[:, 2], vmax=1, vmin=0, s=100)
    for i, txt in enumerate(points):
        ax.annotate(i, (points[i][0].item(), points[i][1].item()), fontsize=10, c='red')
    plt.colorbar(bar1, ax=ax)
eps = 1e-1
boundary = torch.tensor([[-eps,-eps],[-eps, 1. + eps],[1. + eps, 1. + eps],[1. + eps, -eps]], requires_grad=False)

sources = [0]
sinks = [180]
dt=0.0001
n_steps=1000

field = image_to_graph(image)
fig, ax = plt.subplots(figsize=(10, 8))
plot_diagram(field, ax)

In [ ]:
scatter = plt.scatter(field[:, 0], field[:, 1], c=field[:, 2])
#plt.title("Original permeability field (cos(20x) + sin(y))")
# s = list(task1['sources'].keys())
# plt.scatter(x[s], y[s], marker='D', c='m')
ii=0
for j in sinks:
  plt.scatter(field[j,0], field[j,1], s=100, c='g', marker='^')
  plt.annotate('sensor'+str(ii+1), (field[j,0], field[j,1]),xytext=(-20.0, 0),textcoords='offset points', fontsize=16)
  ii+=1

ii=0
for j in sources:
  plt.scatter(field[j,0], field[j,1], s=100, c='r', marker='v')
  plt.annotate('source'+str(ii+1), (field[j,0], field[j,1]), fontsize=16)
  ii+=1
plt.colorbar(scatter)

In [ ]:
storage_loop_01, coarsener_loop_01 = try_different_degrees_of_reduction(field, boundary, dt, n_steps,
                                                    reduction_degrees=[0.1], n_epochs=300,lr=0.01,sigmoid_weight = 0.1, phys_loss_weight = 100,
                                                    aggregating_function=KMeans, method='gnn',
                                                    sources={src: 100 for src in sources}, sinks=sinks,
                                                    denseconv=False,gatconv=True, num_layers=3,temperature=0.4,n_steps_val=0)

In [ ]:
storage_loop_0075, coarsener_loop_0075 = try_different_degrees_of_reduction(field, boundary, dt, n_steps,
                                                    reduction_degrees=[0.075], n_epochs=300,lr=0.01,sigmoid_weight = 1.0, phys_loss_weight = 200,
                                                    aggregating_function=KMeans, method='gnn',
                                                    sources={src: 100 for src in sources}, sinks=sinks,
                                                    denseconv=False,gatconv=True, num_layers=3,temperature=0.4,n_steps_val=0)

In [ ]:
storage_loop_005, coarsener_loop_005 = try_different_degrees_of_reduction(field, boundary, dt, n_steps,
                                                    reduction_degrees=[0.05], n_epochs=300,lr=0.02,sigmoid_weight = 1.0, phys_loss_weight = 300,
                                                    aggregating_function=KMeans, method='gnn',
                                                    sources={src: 100 for src in sources}, sinks=sinks,
                                                    denseconv=False,gatconv=True, num_layers=3,temperature=0.4,n_steps_val=0)

In [ ]:
path='/content/gdrive/MyDrive/ML_ST/new_storage_loop_01.pt'
torch.save(storage_loop_01, path)
path='/content/gdrive/MyDrive/ML_ST/new_coarsener_loop_01.pt'
torch.save(coarsener_loop_01, path)

In [ ]:
path='/content/gdrive/MyDrive/ML_ST/new_coarsener_loop_005.pt'
torch.save(coarsener_loop_005, path)
path='/content/gdrive/MyDrive/ML_ST/new_storage_loop_005.pt'
torch.save(storage_loop_005, path)

In [ ]:


path='/content/gdrive/MyDrive/ML_ST/new_storage_loop_0075.pt'
torch.save(storage_loop_0075, path)

path='/content/gdrive/MyDrive/ML_ST/new_storage_loop_005.pt'
torch.save(storage_loop_005, path)

In [ ]:


path='/content/gdrive/MyDrive/ML_ST/new_coarsener_loop_0075.pt'
torch.save(coarsener_loop_0075, path)

path='/content/gdrive/MyDrive/ML_ST/new_coarsener_loop_005.pt'
torch.save(coarsener_loop_005, path)

In [ ]:
def metis_carsening_loop(path,n_clusters):
    data = []
    with open(path, "r") as file:
        for line in file:
            row = [float(x) for x in line.split()]
            row[-1] = int(row[-1])
            data.append(row)

    data = np.array(data)
    data[0][3]=n_clusters+1
    data[180][3]=n_clusters+2
    clusters = np.unique(data[:, 3])

    centers_of_mass = {}

    for cluster in clusters:
        points = data[data[:, 3] == cluster][:, :3]
        center=[0,0,0]
        points[:,0]=points[:,0]*points[:,2]
        points[:,1]=points[:,1]*points[:,2]
        center[0]=points[:,0].sum()/points[:,2].sum()
        center[1]=points[:,1].sum()/points[:,2].sum()
        center[2]=points[:,2].mean()

        centers_of_mass[cluster] = center

    centers_xy=[]
    for cluster_id, center in centers_of_mass.items():
        centers_xy.append(center)


    #centers_xy.append(field[180])
    #centers_xy.append(field[0])

    centers_xy=np.array(centers_xy)

    x = centers_xy[:,0].reshape(-1, 1)
    y = centers_xy[:,1].reshape(-1, 1)
    k = centers_xy[:,2].reshape(-1, 1)
    eps = 1e-1
    task2 = {'field': np.concatenate([x, y, k], axis=1),
            'sources': {src: 100. for src in [n_clusters]},
            'sinks': [n_clusters+1],
            'dt': 0.00001,
            'boundary': torch.tensor([[-eps,-eps],
                                      [-eps, l + eps],
                                      [l + eps, l + eps],
                                        [l + eps, -eps]], requires_grad=False),
            'n_steps': 1000}

    return task2




In [ ]:
metis_loop_01_field=metis_carsening_loop('/content/output_points_loop_31.txt',31)
metis_loop_0075_field=metis_carsening_loop('/content/output_points_loop_23.txt',23)
metis_loop_005_field=metis_carsening_loop('/content/output_points_loop_15.txt',15)

metis_dynamics_01, ani = dynamics(metis_loop_01_field['field'], sources=metis_loop_01_field['sources'],
                             sinks=metis_loop_01_field['sinks'], boundary=boundary,
                             dt=dt, save_animation=False, n_steps=2*n_steps)
metis_dynamics_0075, ani = dynamics(metis_loop_0075_field['field'], sources=metis_loop_0075_field['sources'],
                             sinks=metis_loop_0075_field['sinks'], boundary=boundary,
                             dt=dt, save_animation=False, n_steps=2*n_steps)
metis_dynamics_005, ani = dynamics(metis_loop_005_field['field'], sources=metis_loop_005_field['sources'],
                             sinks=metis_loop_005_field['sinks'], boundary=boundary,
                             dt=dt, save_animation=False, n_steps=2*n_steps)

In [ ]:
ground_truth_2000_loop, ani = dynamics(field, sources={src: 100 for src in sources},
                             sinks=sinks, boundary=boundary,
                             dt=dt, save_animation=False, n_steps=3*n_steps)

In [ ]:

result_competing_method_loop=torch.load('/content/gdrive/MyDrive/icml_2025/result_competing_method_loop_lr_0.005.pt')

In [ ]:
storage_loop_0075 = torch.load('/content/gdrive/MyDrive/ML_ST/new_storage_loop_0075.pt')

In [ ]:
storage_loop_005 = torch.load('/content/gdrive/MyDrive/ML_ST/new_storage_loop_005.pt')

In [ ]:
storage_loop_01 = torch.load('/content/gdrive/MyDrive/ML_ST/new_storage_loop_01.pt')

In [ ]:
# kmeans
result_kmeans_loop = try_different_degrees_of_reduction(field, boundary, dt, n_steps,
                                                                      reduction_degrees=[0.1, 0.075, 0.05], n_epochs=300,
                                                                      aggregating_function=KMeans, method='kmeans',sources={src: 100 for src in sources}, sinks=sinks,run_wandb=False,clamp_x = [0, 1.0], clamp_y = [0, 1.0])

In [ ]:
ground_truth_2000_loop, ani = dynamics(field, sources={src: 100 for src in sources},
                             sinks=sinks, boundary=boundary,
                             dt=dt, save_animation=False, n_steps=3*n_steps)


orig_curve = ground_truth_2000_loop

result_new_01 = storage_loop_01[0.1]
result_new_005 = storage_loop_005[0.05]
result_new_0075 = storage_loop_0075[0.075]

result_competing_01 = result_competing_method_loop[0.1]
result_competing_005 = result_competing_method_loop[0.05]
result_competing_0075 =  result_competing_method_loop[0.075]

result_kmeans_01 = result_kmeans_loop[0.1]
result_kmeans_005 =result_kmeans_loop[0.05]
result_kmeans_0075 = result_kmeans_loop[0.075]

In [ ]:
result_new = {}
result_new[1] = np.array(orig_curve).flatten()
result_new[0.1] = np.array(result_new_01[0]).flatten()

result_new[0.05] = np.array(result_new_005[0]).flatten()
result_new[0.075] =np.array( result_new_0075[0]).flatten()
result_competing = {}
result_competing[1] = np.array(orig_curve).flatten()
result_competing[0.1] = np.array(result_competing_01[0]).flatten()
result_competing[0.05] = np.array(result_competing_005[0]).flatten()
result_competing[0.075] =np.array(result_competing_0075[0]).flatten()

result_kmeans = {}
result_kmeans[1] = np.array(orig_curve).flatten()
result_kmeans[0.1] = np.array(result_kmeans_01[0]).flatten()
result_kmeans[0.05] = np.array(result_kmeans_005[0]).flatten()
result_kmeans[0.075] = np.array(result_kmeans_0075[0]).flatten()

result_metis={}
result_metis[1] = np.array(orig_curve).flatten()
result_metis[0.1] = metis_dynamics_01[0]
result_metis[0.05] = metis_dynamics_005[0]
result_metis[0.075] = metis_dynamics_0075[0]

In [ ]:
n_steps=1000
ICLR_compare_curves_2(result_new,n_steps)
ICLR_compare_curves_2(result_competing,n_steps)
ICLR_compare_curves_2(result_kmeans,n_steps)
ICLR_compare_curves_2(result_metis,n_steps)

In [ ]:
ICLR_compare_rmse_2([result_competing, result_new, result_kmeans,result_metis], ['Competing algorithm', 'Our algorithm', 'K-Means + AVG','Metis reduction'],0,1000)

In [ ]:
ICLR_compare_rmse_2([result_competing, result_new, result_kmeans,result_metis], ['Competing algorithm', 'Our algorithm', 'K-Means + AVG','Metis reduction'],1000,2000)

##DMD loop

In [ ]:
def ICLR_compare_curves_different_reduction_algs(result,labels,title='',steps_split=1000):
    fig, ax = plt.subplots(figsize=(14, 10))
    for k, v in result.items():
        ax.plot(v, label=labels[k], linewidth=4)

    split_point = steps_split
    ax.axvline(x=split_point, color='r', linestyle='--')
    y_min, y_max = ax.get_ylim()
    ax.text(split_point, y_max*0.9, 'Train/Test Split', rotation=0, verticalalignment='center', color='black',fontsize=24)

    plt.legend(fontsize=20)
    ax.set_ylabel('$p_{s},~Pa$', fontsize=20)
    ax.set_xlabel('time steps', fontsize=20)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.title(title,fontsize = 20)
    plt.grid(True)
    plt.show()
# result={}
# result[0]=x[sinks[0][0],:500]
# result[1]=ground_truth_1[task1['sinks'][0],:500]
# result[2]=storage_01[0.1][0][0].tolist()[:500]
# ICLR_compare_curves_different_reduction_algs(result,["DMD","ground truth","our algorithm"],"sinusoidal permeability field")

result={}
result[0]=x[sinks[0]]
result[1]=ground_truth_loop[sinks[0]]
result[2]=storage_loop_01[0.1][0][0].tolist()[:2000]
ICLR_compare_curves_different_reduction_algs(result,["DMD","ground truth","our algorithm"],"loop scenario")

## Checking generalizing ability on another grid

In [ ]:
def inference_new(field,boundary,coarsener, dt=0.0001,n_steps=100,sources=None,sinks=None,num_layers=1, clusters=900,ground_truth=None):

    ground_truth, _ = dynamics(field, sources={src: 100 for src in sources},
                             sinks=sinks, boundary=boundary,
                             dt=dt, save_animation=False, n_steps=n_steps)
    FIELD = torch.tensor(field, requires_grad=False, dtype=torch.float)

    mesh = triangulate(FIELD[:, :2].clone().detach())
    #sources = list(sources.keys())

    with torch.no_grad():
      edge_index, areas, e, clipped_vertices_dict,_ = differentiable_voronoi(FIELD[:, :2], mesh, boundary)
      h = calc_h(FIELD[:, :2], edge_index)
      #print(perm.shape, h.shape, e.shape, edge_index.shape)

      edge_weights = calc_physics_weights(FIELD[:, 2], e, h, edge_index)
      edge_index, edge_weights = to_undirected(edge_index, edge_weights)
      print('shape edge_weights ',edge_weights.shape)
      print('shape edge_index ',edge_index.shape)
      s, out,_ = coarsener(torch.unsqueeze(FIELD, 0), edge_index, edge_weights)

    sources_reduced = list(range(len(out[0]) - len(sources),  len(out[0])))
    sources_reduced = {src: 100 for src in sources_reduced}
    sinks_reduced = list(range(len(out[0]) - len(sources) - len(sinks),  len(out[0]) - len(sources)))
    print(sources_reduced, sinks_reduced)

    print(out[0].shape)

    reduced_dynamics, _ = dynamics(out[0], {src: 100 for src in sources_reduced},sinks_reduced, boundary, n_steps=n_steps, dt = dt)
    print(reduced_dynamics.shape)

    return reduced_dynamics,ground_truth,out[0]

In [ ]:
def inference_new_sinks(field,boundary,model_dict, dt=0.0001,n_steps=100,sources=None,sinks=None,num_layers=3, clusters=900,denseconv=False,gatconv=True):

    ground_truth, _ = dynamics(field, sources={src: 100 for src in sources},
                             sinks=sinks, boundary=boundary,
                             dt=dt, save_animation=False, n_steps=n_steps)
    FIELD = torch.tensor(field, requires_grad=False, dtype=torch.float)

    mesh = triangulate(FIELD[:, :2].clone().detach())
    #sources = list(sources.keys())
    #coarsener=learnableCoarsening(channels=[3,8],k=clusters,sources=sources,sinks=sinks,temperature=0.4,denseconv=denseconv,gatconv=gatconv,num_layers=num_layers)
    #coarsener.load_state_dict(model_dict)
    with torch.no_grad():
      edge_index, areas, e, clipped_vertices_dict,boundary_nodes = differentiable_voronoi(FIELD[:, :2], mesh, boundary)
      h = calc_h(FIELD[:, :2], edge_index)
      #print(perm.shape, h.shape, e.shape, edge_index.shape)

      edge_weights = calc_physics_weights(FIELD[:, 2], e, h, edge_index)
      edge_index, edge_weights = to_undirected(edge_index, edge_weights)
      print('shape edge_weights ',edge_weights.shape)
      print('shape edge_index ',edge_index.shape)

      coarsener=learnableCoarsening(channels=[3,8],k=clusters,sources={src: 100 for src in sources},sinks=sinks,temperature=0.4,denseconv=denseconv,gatconv=gatconv,num_layers=num_layers,boundary_nodes=list(boundary_nodes))
      coarsener.load_state_dict(model_dict)
      s, out,_ = coarsener(torch.unsqueeze(FIELD, 0), edge_index, edge_weights)

    sources_reduced = list(range(len(out[0]) - len(sources),  len(out[0])))
    sources_reduced = {src: 100 for src in sources_reduced}
    sinks_reduced = list(range(len(out[0]) - len(sources) - len(sinks),  len(out[0]) - len(sources)))
    print(sources_reduced, sinks_reduced)

    print(out[0].shape)

    reduced_dynamics, _ = dynamics(out[0], {src: 100 for src in sources_reduced},sinks_reduced, boundary, n_steps=n_steps, dt = dt)
    print(reduced_dynamics.shape)

    mesh = triangulate(out[0][:, :2].clone().detach())
    edge_index, areas, e, coarsened_clipped_vertices_dict,boundary_nodes = differentiable_voronoi(out[0][:, :2], mesh, boundary)

    fig, ax1 = plt.subplots(figsize=(14, 10))
    for vertex in coarsened_clipped_vertices_dict.keys():
        line = coarsened_clipped_vertices_dict[vertex].clone().detach()
        cmap = plt.get_cmap('RdYlBu')
        p = matplotlib.patches.Polygon(line, facecolor = 'g', edgecolor='black', alpha=0.2, fill=False)
        m = torch.mean(torch.unique(line, dim=0), dim=0)
        ax1.add_patch(p)
    scale = ax1.scatter(out[0].detach()[:, 0], out[0].detach()[:, 1], c=out[0].detach().numpy()[:, 2], s=400)
    if sources is not None:
        for j, src in enumerate(sources_reduced):
            ax1.scatter(out[0].detach()[src, 0].item(), out[0].detach()[src, 1].item(), s=100, c='r', marker='v')
            ax1.text(out[0].detach()[src, 0].item(), out[0].detach()[src, 1].item(), 'source ' + str(j+1), fontsize=30)

    if sinks is not None:
        for j, snk in enumerate(sinks_reduced):
            ax1.scatter(out[0].detach()[snk, 0].item(), out[0].detach()[snk, 1].item(), s=100, c='g', marker='^')
            ax1.text(out[0].detach()[snk, 0].item(), out[0].detach()[snk, 1].item(), 'sensor ' + str(j+1), fontsize=30)
    cbar = plt.colorbar(scale)
    cbar.ax.tick_params(labelsize=24)
    cbar.ax.set_ylabel('$K(x, y)$', fontsize=40)

    return reduced_dynamics,ground_truth,out[0]

In [ ]:
def ICLR_compare_curves_inference(result,title=""):
    fig, ax = plt.subplots(figsize=(14, 10))
    for k, v in result.items():
        ax.plot(v[0], label=str(k), linewidth=4)
    plt.legend(fontsize=20)
    ax.set_ylabel('$p_{s},~Pa$', fontsize=20)
    ax.set_xlabel('time steps', fontsize=20)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.gca().set_title(title,fontsize = 20)
    plt.grid(True)
    plt.show()

In [ ]:
coarsener_loop_01 = torch.load('/content/gdrive/MyDrive/ML_ST/new_coarsener_loop_01.pt',weights_only=False)
#coarsener_loop_0075 = coar_loop[0.075]

In [ ]:
coarsener_loop_0075

In [ ]:
reduced_dynamics_graphconv_179,ground_truth_graphconv_179,out_179=inference_new_sinks(field,boundary,coarsener_loop_0075[0.075].state_dict(),clusters = 23,dt=0.0001,n_steps=1000,sources=[0],sinks=[179])
reduced_dynamics_graphconv_153,ground_truth_graphconv_153,out_153=inference_new_sinks(field,boundary,coarsener_loop_0075[0.075].state_dict(),clusters = 23,dt=0.0001,n_steps=1000,sources=[0],sinks=[128])
reduced_dynamics_graphconv_101,ground_truth_graphconv_101,out_101=inference_new_sinks(field,boundary,coarsener_loop_0075[0.075].state_dict(),clusters = 23,dt=0.0001,n_steps=1000,sources=[0],sinks=[101])

result_new_179 = {}
result_new_153 = {}
result_new_101 = {}
result_new_179['new_sink_1,gt'] = ground_truth_graphconv_179
result_new_153['new_sink_2,gt'] = ground_truth_graphconv_153
result_new_101['new_sink_3,gt'] = ground_truth_graphconv_101

result_new_179['reduced'] = reduced_dynamics_graphconv_179.tolist()
result_new_153['reduced'] = reduced_dynamics_graphconv_153.tolist()
result_new_101['reduced'] = reduced_dynamics_graphconv_101.tolist()


In [ ]:
field[179]

In [ ]:
field[177]

In [ ]:
field[101]

In [ ]:
reduced_dynamics_graphconv_179,ground_truth_graphconv_179,out_179=inference_new_sinks(field,boundary,coarsener_loop_01[0.1].state_dict(),clusters = 31,dt=0.0001,n_steps=1000,sources=[0],sinks=[179])
reduced_dynamics_graphconv_153,ground_truth_graphconv_153,out_153=inference_new_sinks(field,boundary,coarsener_loop_01[0.1].state_dict(),clusters = 31,dt=0.0001,n_steps=1000,sources=[0],sinks=[177])
reduced_dynamics_graphconv_101,ground_truth_graphconv_101,out_101=inference_new_sinks(field,boundary,coarsener_loop_01[0.1].state_dict(),clusters = 31,dt=0.0001,n_steps=1000,sources=[0],sinks=[101])

result_new_179 = {}
result_new_153 = {}
result_new_101 = {}
result_new_179['new_sensor_1_gt'] = ground_truth_graphconv_179
result_new_153['new_sensor_2_gt'] = ground_truth_graphconv_153
result_new_101['new_sensor_3_gt'] = ground_truth_graphconv_101

result_new_179['reduced_sensor_1'] = reduced_dynamics_graphconv_179.tolist()
result_new_153['reduced_sensor_2'] = reduced_dynamics_graphconv_153.tolist()
result_new_101['reduced_sensor_3'] =reduced_dynamics_graphconv_101.tolist()


In [ ]:
ICLR_compare_curves_inference(result_new_179)
ICLR_compare_curves_inference(result_new_153)
ICLR_compare_curves_inference(result_new_101)

##Fine tuning decagon

In [ ]:
seed=42
set_seeds(seed)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import torch

cx, cy = 2, 2
r = 1
num_points = 400

angles = np.linspace(-2 * np.pi, 0, 10, endpoint=False)
vertices = np.array([cx + r * np.cos(angles), cy + r * np.sin(angles)]).T

def is_inside_octagon(x, y, vertices):
    point = Point(x, y)
    polygon = Polygon(vertices)
    return polygon.contains(point)

def generate_points_in_octagon(num_points, vertices):
    points = []
    while len(points) < num_points:
        x, y = np.random.uniform(1, 3), np.random.uniform(1, 3)
        if is_inside_octagon(x, y, vertices):
            points.append((x, y))
    return np.array(points)

num_points = 400
points = generate_points_in_octagon(num_points, vertices)


In [ ]:
def hold_points_in_octagon(x,y, vertices,N):
    points = []
    for i in range(N):
        if is_inside_octagon(x[i][0].item(), y[i][0].item(),vertices):
            points.append((x[i][0].item(), y[i][0].item()))
    return np.array(points)

In [ ]:
eps = 1e-1
vertices_boundary = np.array([cx + (r+eps) * np.cos(angles), cy + (r+eps) * np.sin(angles)]).T[::-1]
vertices_boundary

In [ ]:

l = 3.0
nx = 25

x, y = np.meshgrid(np.linspace(1, l, nx), np.linspace(1, l, nx))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)

points=hold_points_in_octagon(x,y, vertices,nx*nx)
print(points.shape)
x,y=points[:, 0], points[:, 1]

print(x.shape)
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)

k = np.cos(20 * x)+ np.sin(y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task1 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [373]},
        #'sinks': [21,32,395,406,224,206],
        'sinks': [21],
        'dt': 0.0001,
        # 'boundary': torch.tensor([[-eps,-eps],
        #                           [-eps, l + eps],
        #                            [l + eps, l + eps],
        #                             [l + eps, -eps]], requires_grad=False),
        'boundary': torch.tensor(vertices_boundary.copy(), requires_grad=False),
        'n_steps': 1000}

In [ ]:
scatter = plt.scatter(task1['field'][:, 0], task1['field'][:, 1], c=task1['field'][:, 2])
plt.title("Original permeability field (cos(20x) + sin(y))")
plt.colorbar(scatter)

In [ ]:
ground_truth, ani = dynamics(task1['field'], sources=task1['sources'],
                             sinks=task1['sinks'], boundary=task1['boundary'],
                             dt=task1['dt'], save_animation=False, n_steps=task1['n_steps'])

In [ ]:
plt.plot(ground_truth[0])

In [ ]:
import matplotlib.cm as cm


scatter = plt.scatter(task1['field'][:, 0], task1['field'][:, 1], c=task1['field'][:, 2], norm="log", cmap=cm.jet)
#plt.clim(1e-3, 1e+3)
plt.scatter(x[task1['sinks']], y[task1['sinks']], marker='D', c='k')
s = list(task1['sources'].keys())
plt.scatter(x[s], y[s], marker='D', c='m')
ii=0
for j in task1['sinks']:
  plt.annotate(str(ii+1), (task1['field'][j,0], task1['field'][j,1]), fontsize=16)
  ii+=1

oct_x, oct_y = vertices[:,0],vertices[:,1]
#plt.fill(oct_x, oct_y, 'lightblue', alpha=0.5)
plt.plot(vertices[:, 0], vertices[:, 1], 'r-',color='black')
plt.plot([oct_x[9],oct_x[0]], [oct_y[9],oct_y[0]], color='black')

plt.title("Permeability")
plt.colorbar(scatter)


In [ ]:
seed=42
set_seeds(seed)
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task1['field'],
                                                                    sources=task1['sources'],
                                                                    sinks=task1['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task1['boundary'],
                                                                    dt=task1['dt'], n_steps=task1['n_steps'], n_epochs=300,phys_loss_weight=500.0,lr=0.015,sigmoid_weight=1.0,clusters=40,
                                                                    denseconv=False, num_layers=3,temperature=0.4,n_steps_val=0,warmup_steps=50,gatconv = True,seed=seed)

In [ ]:
torch.save(optimized_dmon, "/content/gdrive/MyDrive/ML_ST/optimized_dmon_full_clusters_40_decagon.pt")
torch.save(loss_curve, "/content/gdrive/MyDrive/ML_ST/loss_curve_clusters_40_decagon.pt")
#torch.save(optimizer, "/content/gdrive/MyDrive/ML_ST/optimizer_clusters_40_ten_angles.pt")
torch.save(optimized_coords, "/content/gdrive/MyDrive/ML_ST/optimized_coords_clusters_40_decagon.pt")

In [ ]:
seed=42
set_seeds(seed)
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task1['field'],
                                                                    sources=task1['sources'],
                                                                    sinks=task1['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task1['boundary'],
                                                                    dt=task1['dt'], n_steps=task1['n_steps'], n_epochs=300,phys_loss_weight=500.0,lr=0.02,sigmoid_weight=1.0,clusters=40,
                                                                    denseconv=False, num_layers=2,temperature=0.4,n_steps_val=0,warmup_steps=50,gatconv = True,seed=seed)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
torch.save(optimized_dmon, "/content/gdrive/MyDrive/ML_ST/optimized_dmon_full_clusters_40_ten_angles.pt")
torch.save(loss_curve, "/content/gdrive/MyDrive/ML_ST/loss_curve_clusters_40_ten_angles.pt")
#torch.save(optimizer, "/content/gdrive/MyDrive/ML_ST/optimizer_clusters_40_ten_angles.pt")
torch.save(optimized_coords, "/content/gdrive/MyDrive/ML_ST/optimized_coords_clusters_40_ten_angles.pt")

In [ ]:

l = 3.0
nx = 25
starting_point=1.0


x, y = np.meshgrid(np.linspace(starting_point, l, nx), np.linspace(starting_point, l, nx))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)

#points=hold_points_in_octagon(x,y, vertices,nx*nx)
# print(points.shape)
# x,y=points[:, 0], points[:, 1]

# print(x.shape)
# x = x.flatten().reshape(-1, 1)
# y = y.flatten().reshape(-1, 1)

k = np.cos(20 * x)+ np.sin(y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task2 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [18+25*19]},
        #'sinks': [21,32,395,406,224,206],
        'sinks': [6+25*3],
        'dt': 0.0001,
        'boundary': torch.tensor([[starting_point-eps,starting_point-eps],
                                  [starting_point-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, starting_point-eps]], requires_grad=False),
        #'boundary': torch.tensor(vertices_boundary.copy(), requires_grad=False),
        'n_steps': 1000}

In [ ]:
FIELD = torch.tensor(task2['field'], requires_grad=False, dtype=torch.float)
mesh = triangulate(FIELD[:, :2].clone().detach().numpy())
#print(FIELD.shape, boundary.shape)
edge_index, areas, e, clipped_vertices_dict,boundary_nodes = differentiable_voronoi(FIELD[:, :2], mesh, task2['boundary'])

In [ ]:
ground_truth, ani = dynamics(task2['field'], sources=task2['sources'],
                             sinks=task2['sinks'], boundary=task2['boundary'],
                             dt=task2['dt'], save_animation=False, n_steps=task2['n_steps'])


In [ ]:
coarsener_decagon=torch.load("/content/gdrive/MyDrive/ML_ST/optimized_dmon_full_clusters_40_decagon.pt",weights_only=False)
optimized_dmon=coarsener_decagon

In [ ]:
seed =42
set_seeds(seed)
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task2['field'],
                                                                    sources=task2['sources'],
                                                                    sinks=task2['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task2['boundary'],
                                                                    dt=task2['dt'], n_steps=task1['n_steps'], n_epochs=20,phys_loss_weight=1000,lr=0.02,sigmoid_weight=1.0,
                                                                    denseconv=False,num_layers=3,temperature=0.4,n_steps_val = 0,clusters=40,seed=seed,
                                                                    coarsener=optimized_dmon,warmup_steps=0,
                                                                    gatconv = True)

In [ ]:
seed = 42
set_seeds(seed)
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task2['field'],
                                                                    sources=task2['sources'],
                                                                    sinks=task2['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task2['boundary'],
                                                                    dt=task2['dt'], n_steps=task1['n_steps'], n_epochs=10,phys_loss_weight=500,lr=0.02,sigmoid_weight=1.0,
                                                                    denseconv=False,num_layers=3,temperature=0.4,n_steps_val = 0,clusters=40,seed=seed,
                                                                    coarsener=optimized_dmon,warmup_steps=0,
                                                                    gatconv = True)

In [ ]:
seed =42
set_seeds(seed)
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task2['field'],
                                                                    sources=task2['sources'],
                                                                    sinks=task2['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task2['boundary'],
                                                                    dt=task2['dt'], n_steps=task1['n_steps'], n_epochs=20,phys_loss_weight=500,lr=0.01,sigmoid_weight=1.0,
                                                                    denseconv=False,num_layers=3,temperature=0.4,n_steps_val = 0,clusters=40,seed=seed,
                                                                    coarsener=optimized_dmon,warmup_steps=0,
                                                                    gatconv = True)

In [ ]:
seed =42
set_seeds(seed)
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task2['field'],
                                                                    sources=task2['sources'],
                                                                    sinks=task2['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task2['boundary'],
                                                                    dt=task2['dt'], n_steps=task1['n_steps'], n_epochs=20,phys_loss_weight=500,lr=0.02,sigmoid_weight=1.0,
                                                                    denseconv=False,num_layers=3,temperature=0.4,n_steps_val = 0,clusters=40,seed=seed,
                                                                    coarsener=optimized_dmon,warmup_steps=0,
                                                                    gatconv = True)

In [ ]:
seed = 42
set_seeds(seed)
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task2['field'],
                                                                    sources=task2['sources'],
                                                                    sinks=task2['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task2['boundary'],
                                                                    dt=task2['dt'], n_steps=task1['n_steps'], n_epochs=20,phys_loss_weight=500,lr=0.02,sigmoid_weight=1.0,
                                                                    denseconv=False,num_layers=2,temperature=0.4,n_steps_val = 0,clusters=40,seed=seed,
                                                                    coarsener=optimized_dmon,warmup_steps=0,
                                                                    gatconv = True)

##Fine tune other field

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
l = 1
x, y = np.meshgrid(np.linspace(0, l, 20), np.linspace(0, l, 20))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)
k = np.cos(20 * x)+ np.sin(5*y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task2 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [210]},
        'sinks': [42, 399-42 ],
        'dt': 0.00001,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 1000}

In [ ]:
ground_truth, ani = dynamics(task2['field'], sources=task2['sources'],
                             sinks=task2['sinks'], boundary=task2['boundary'],
                             dt=task2['dt'], save_animation=False, n_steps=task2['n_steps'])


In [ ]:
coarsener_sin=torch.load("/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_01.pt",weights_only=False)
optimized_dmon=coarsener_sin[0.1]

In [ ]:
seed =42
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task2['field'],
                                                                    sources=task2['sources'],
                                                                    sinks=task2['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task2['boundary'],
                                                                    dt=task2['dt'], n_steps=task2['n_steps'], n_epochs=20,phys_loss_weight=500,lr=0.01,sigmoid_weight=1.0,
                                                                    denseconv=False,num_layers=3,temperature=0.4,n_steps_val = 0,clusters=40,
                                                                    seed=seed,coarsener=optimized_dmon,warmup_steps=0)

In [ ]:
seed =42
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task2['field'],
                                                                    sources=task2['sources'],
                                                                    sinks=task2['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task2['boundary'],
                                                                    dt=task2['dt'], n_steps=task2['n_steps'], n_epochs=20,phys_loss_weight=500,lr=0.01,sigmoid_weight=1.0,
                                                                    denseconv=False,num_layers=3,temperature=0.4,n_steps_val = 0,clusters=40,
                                                                    seed=seed,coarsener=optimized_dmon,warmup_steps=0)

In [ ]:

loss_curve, optimized_dmon, adjacency, optimized_coords = train(task2['field'],
                                                                    sources=task2['sources'],
                                                                    sinks=task2['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task2['boundary'],
                                                                    dt=task2['dt'], n_steps=task2['n_steps'], n_epochs=20,phys_loss_weight=500,lr=0.01,sigmoid_weight=1.0,
                                                                    denseconv=False,num_layers=3,temperature=0.4,n_steps_val = 0,clusters=40,
                                                                    seed=seed,coarsener=optimized_dmon,warmup_steps=0)

In [ ]:
reduced_dynamics_2,gt_2,coarsened_point_cloud_2 = inference_new(task2['field'], task2['boundary'],optimized_dmon, dt=task2['dt'],n_steps=10000,sources=list(task2['sources'].keys()),sinks=task2['sinks'], clusters = 40,ground_truth=ground_truth)

In [ ]:
rmse_1 = torch.sqrt(torch.mean((reduced_dynamics_2[0]-gt_2[0])**2))
rmse_2 = torch.sqrt(torch.mean((reduced_dynamics_2[1]-gt_2[1])**2))


plt.plot(reduced_dynamics_2[0], label='reduced_sensor_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
plt.plot(reduced_dynamics_2[1], label='reduced_sensor_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
plt.plot(gt_2[0], label = 'gt1')
plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('p(t)')
plt.title('Curves on cos(20*x) + sin(5*y)')
plt.legend()
plt.grid(True)
plt.plot()

In [ ]:
plt.plot(reduced_dynamics_2[0], label='reduced_sensor_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
#plt.plot(reduced_dynamics_2[1], label='reduced_sink_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
plt.plot(gt_2[0], label = 'gt1')
#plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('$p_{s}.~Pa$')
plt.title('Curves on cos(20*x) + sin(5*y)')
plt.legend()
plt.grid(True)
plt.plot()

In [ ]:
#plt.plot(reduced_dynamics_2[0], label='reduced_sink_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
plt.plot(reduced_dynamics_2[1], label='reduced_sensor_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
#plt.plot(gt_2[0], label = 'gt1')
plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('$p_{s}.~Pa$')
plt.title('Curves on cos(20*x) + sin(5*y)')
plt.legend()
plt.grid(True)
plt.plot()

In [ ]:
l = 1
x, y = np.meshgrid(np.linspace(0, l, 20), np.linspace(0, l, 20))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)
k = np.cos(15 * x)+ np.sin(5*y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task2 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [210]},
        'sinks': [42, 399-42 ],
        'dt': 0.00001,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 1000}

In [ ]:
ground_truth, ani = dynamics(task2['field'], sources=task2['sources'],
                             sinks=task2['sinks'], boundary=task2['boundary'],
                             dt=task2['dt'], save_animation=False, n_steps=task2['n_steps'])


In [ ]:
coarsener_sin=torch.load("/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_01.pt",weights_only=False)
optimized_dmon=coarsener_sin[0.1]

In [ ]:
seed=42
loss_curve, optimized_dmon, adjacency, optimized_coords = train(task2['field'],
                                                                    sources=task2['sources'],
                                                                    sinks=task2['sinks'],
                                                                    ground_truth_dynamics=ground_truth,
                                                                    boundary=task2['boundary'],
                                                                    dt=task2['dt'], n_steps=task2['n_steps'], n_epochs=20,phys_loss_weight=500,lr=0.01,sigmoid_weight=1.0,
                                                                    denseconv=False,num_layers=3,temperature=0.4,n_steps_val = 0,clusters=40,
                                                                    seed=seed,coarsener=optimized_dmon,warmup_steps=0)

In [ ]:
reduced_dynamics_2,gt_2,coarsened_point_cloud_2 = inference_new(task2['field'], task2['boundary'],optimized_dmon, dt=task2['dt'],n_steps=10000,sources=list(task2['sources'].keys()),sinks=task2['sinks'], clusters = 40,ground_truth=ground_truth)

In [ ]:
rmse_1 = torch.sqrt(torch.mean((reduced_dynamics_2[0]-gt_2[0])**2))
rmse_2 = torch.sqrt(torch.mean((reduced_dynamics_2[1]-gt_2[1])**2))


plt.plot(reduced_dynamics_2[0], label='reduced_sensor_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
plt.plot(reduced_dynamics_2[1], label='reduced_sensor_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
plt.plot(gt_2[0], label = 'gt1')
plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('$p_{s}.~Pa$')
plt.title('Curves on cos(15*x) + sin(5*y)')
plt.legend()
plt.grid(True)
plt.plot()

In [ ]:
rmse_1 = torch.sqrt(torch.mean((reduced_dynamics_2[0]-gt_2[0])**2))
rmse_2 = torch.sqrt(torch.mean((reduced_dynamics_2[1]-gt_2[1])**2))

plt.plot(reduced_dynamics_2[0], label='reduced_sensor_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
#plt.plot(reduced_dynamics_2[1], label='reduced_sink_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
plt.plot(gt_2[0], label = 'gt1')
#plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('$p_{s}.~Pa$')
plt.title('Curves on cos(15*x) + sin(5*y)')
plt.legend()
plt.grid(True)
plt.plot()

In [ ]:
#plt.plot(reduced_dynamics_2[0], label='reduced_sink_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
plt.plot(reduced_dynamics_2[1], label='reduced_sensor_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
#plt.plot(gt_2[0], label = 'gt1')
plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('$p_{s}.~Pa$')
plt.title('Curves on cos(15*x) + sin(5*y)')
plt.legend()
plt.grid(True)
plt.plot()

In [ ]:
l = 1
x, y = np.meshgrid(np.linspace(0, l, 20), np.linspace(0, l, 20))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)
k = np.cos(19 * x)+ np.sin(y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task2 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [210]},
        'sinks': [42, 399-42 ],
        'dt': 0.00001,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 1000}

ground_truth, ani = dynamics(task2['field'], sources=task2['sources'],
                             sinks=task2['sinks'], boundary=task2['boundary'],
                             dt=task2['dt'], save_animation=False, n_steps=task2['n_steps'])



In [ ]:
coarsener_sin=torch.load("/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_01.pt",weights_only=False)
optimized_dmon=coarsener_sin[0.1]

In [ ]:
reduced_dynamics_2,gt_2,coarsened_point_cloud_2 = inference_new(task2['field'], task2['boundary'],optimized_dmon, dt=task2['dt'],n_steps=1000,sources=list(task2['sources'].keys()),sinks=task2['sinks'], clusters = 40,ground_truth=ground_truth)

rmse_1 = torch.sqrt(torch.mean((reduced_dynamics_2[0]-gt_2[0])**2))
rmse_2 = torch.sqrt(torch.mean((reduced_dynamics_2[1]-gt_2[1])**2))

plt.plot(reduced_dynamics_2[0], label='reduced_sensor_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
plt.plot(reduced_dynamics_2[1], label='reduced_sensor_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
plt.plot(gt_2[0], label = 'gt1')
plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('$p_{s},~Pa$')
plt.title('Curves on cos(19*x) + sin(y)')
plt.legend()
plt.grid(True)
plt.plot()


In [ ]:
l = 1
x, y = np.meshgrid(np.linspace(0, l, 20), np.linspace(0, l, 20))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)
k = np.cos(21 * x)+ np.sin(y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task2 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [210]},
        'sinks': [42, 399-42 ],
        'dt': 0.00001,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 1000}

ground_truth, ani = dynamics(task2['field'], sources=task2['sources'],
                             sinks=task2['sinks'], boundary=task2['boundary'],
                             dt=task2['dt'], save_animation=False, n_steps=task2['n_steps'])
coarsener_sin=torch.load("/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_01.pt",weights_only=False)
optimized_dmon=coarsener_sin[0.1]

reduced_dynamics_2,gt_2,coarsened_point_cloud_2 = inference_new(task2['field'], task2['boundary'],optimized_dmon, dt=task2['dt'],n_steps=1000,sources=list(task2['sources'].keys()),sinks=task2['sinks'], clusters = 40,ground_truth=ground_truth)

rmse_1 = torch.sqrt(torch.mean((reduced_dynamics_2[0]-gt_2[0])**2))
rmse_2 = torch.sqrt(torch.mean((reduced_dynamics_2[1]-gt_2[1])**2))

plt.plot(reduced_dynamics_2[0], label='reduced_sensor_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
plt.plot(reduced_dynamics_2[1], label='reduced_sensor_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
plt.plot(gt_2[0], label = 'gt1')
plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('$p_{s},~Pa$')
plt.title('Curves on cos(21*x) + sin(y)')
plt.legend()
plt.grid(True)
plt.plot()


In [ ]:
l = 1
x, y = np.meshgrid(np.linspace(0, l, 20), np.linspace(0, l, 20))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)
k = np.cos(20 * x)+ np.sin(1.5*y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task2 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [210]},
        'sinks': [42, 399-42 ],
        'dt': 0.00001,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 1000}

ground_truth, ani = dynamics(task2['field'], sources=task2['sources'],
                             sinks=task2['sinks'], boundary=task2['boundary'],
                             dt=task2['dt'], save_animation=False, n_steps=task2['n_steps'])
coarsener_sin=torch.load("/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_01.pt",weights_only=False)
optimized_dmon=coarsener_sin[0.1]

reduced_dynamics_2,gt_2,coarsened_point_cloud_2 = inference_new(task2['field'], task2['boundary'],optimized_dmon, dt=task2['dt'],n_steps=1000,sources=list(task2['sources'].keys()),sinks=task2['sinks'], clusters = 40,ground_truth=ground_truth)

rmse_1 = torch.sqrt(torch.mean((reduced_dynamics_2[0]-gt_2[0])**2))
rmse_2 = torch.sqrt(torch.mean((reduced_dynamics_2[1]-gt_2[1])**2))

plt.plot(reduced_dynamics_2[0], label='reduced_sensor_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
plt.plot(reduced_dynamics_2[1], label='reduced_sensor_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
plt.plot(gt_2[0], label = 'gt1')
plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('$p_{s},~Pa$')
plt.title('Curves on cos(20*x) + sin(1.5*y)')
plt.legend()
plt.grid(True)
plt.plot()


In [ ]:
l = 1
x, y = np.meshgrid(np.linspace(0, l, 20), np.linspace(0, l, 20))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)
k = np.cos(20 * x)+ np.sin(2.0*y)
k = k + np.abs(np.min(k)) + 2.0
eps = 1e-1
task2 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [210]},
        'sinks': [42, 399-42 ],
        'dt': 0.00001,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 1000}

ground_truth, ani = dynamics(task2['field'], sources=task2['sources'],
                             sinks=task2['sinks'], boundary=task2['boundary'],
                             dt=task2['dt'], save_animation=False, n_steps=task2['n_steps'])
coarsener_sin=torch.load("/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_01.pt",weights_only=False)
optimized_dmon=coarsener_sin[0.1]

reduced_dynamics_2,gt_2,coarsened_point_cloud_2 = inference_new(task2['field'], task2['boundary'],optimized_dmon, dt=task2['dt'],n_steps=1000,sources=list(task2['sources'].keys()),sinks=task2['sinks'], clusters = 40,ground_truth=ground_truth)

rmse_1 = torch.sqrt(torch.mean((reduced_dynamics_2[0]-gt_2[0])**2))
rmse_2 = torch.sqrt(torch.mean((reduced_dynamics_2[1]-gt_2[1])**2))

plt.plot(reduced_dynamics_2[0], label='reduced_sensor_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
plt.plot(reduced_dynamics_2[1], label='reduced_sensor_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
plt.plot(gt_2[0], label = 'gt1')
plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('$p_{s},~Pa$')
plt.title('Curves on cos(20*x) + sin(2.0*y)')
plt.legend()
plt.grid(True)
plt.plot()


In [ ]:
ground_truth, ani = dynamics(task2['field'], sources=task2['sources'],
                             sinks=task2['sinks'], boundary=task2['boundary'],
                             dt=task2['dt'], save_animation=False, n_steps=task2['n_steps'])


In [ ]:
coarsener_sin=torch.load("/content/gdrive/MyDrive/ML_ST/new_coarsener_sin_01.pt",weights_only=False)
optimized_dmon=coarsener_sin[0.1]

In [ ]:
reduced_dynamics_2,gt_2,coarsened_point_cloud_2 = inference_new(task2['field'], task2['boundary'],optimized_dmon, dt=task2['dt'],n_steps=1000,sources=list(task2['sources'].keys()),sinks=task2['sinks'], clusters = 40,ground_truth=ground_truth)

In [ ]:
plt.plot(reduced_dynamics_2[0], label='reduced_sink_1' + " RMSE: {:1.4f}".format(rmse_1.item()))
plt.plot(reduced_dynamics_2[1], label='reduced_sink_2' + " RMSE: {:1.4f}".format(rmse_2.item()))
plt.plot(gt_2[0], label = 'gt1')
plt.plot(gt_2[1], label = 'gt2')
plt.xlabel('timestep')
plt.ylabel('p(t)')
plt.title('Curves on cos(15*x) + sin(5*y)')
plt.legend()
plt.grid(True)
plt.plot()

## Log uniform

In [ ]:
from scipy.stats import loguniform
set_seeds(40)
l = 1
x, y = np.meshgrid(np.linspace(0, l, 30), np.linspace(0, l, 30))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)

k = np.zeros(x.shape)
q=0
while q < len(x):
    k[q] = loguniform.rvs(0.1, 10)
    if k[q] > 0.1 and k[q]< 10:
        q+=1

eps = 1e-1
task1 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [465]},
        'sinks': [63],
        'dt': 0.00005,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 1000}

In [ ]:
scatter = plt.scatter(task1['field'][:, 0], task1['field'][:, 1], c=task1['field'][:, 2])
plt.title("Original permeability field")
plt.colorbar(scatter)

In [ ]:
ground_truth, ani = dynamics(task1['field'], sources=task1['sources'],
                             sinks=task1['sinks'], boundary=task1['boundary'],
                             dt=task1['dt'], save_animation=False, n_steps=task1['n_steps'])
plt.plot(ground_truth[0])


In [ ]:
result_competing_method = try_different_degrees_of_reduction(task1['field'], task1['boundary'], task1['dt'], task1['n_steps'], reduction_degrees=[0.1], n_epochs=300, aggregating_function=KMeans, method='points',
                                                             sources = task1['sources'], sinks = task1['sinks'],clamp_x = [0, 1.0], clamp_y = [0, 1.0], lr = 0.001,save_ani=False)

In [ ]:
storage, coarsener = try_different_degrees_of_reduction(task1['field'], task1['boundary'], task1['dt'], task1['n_steps'],
                                                        reduction_degrees=[0.1], n_epochs=700, lr = 0.02, sigmoid_weight = 1.0, phys_loss_weight = 500,
                                                        aggregating_function=KMeans, method='gnn', sources = task1['sources'], sinks = task1['sinks'], save_ani=False,
                                                        denseconv=False, num_layers=3,temperature=0.4,n_steps_val=0,warmup_steps=50,gatconv = True)


In [ ]:
from scipy.stats import loguniform
set_seeds(41)
l = 1
x, y = np.meshgrid(np.linspace(0, l, 30), np.linspace(0, l, 30))
x = x.flatten().reshape(-1, 1)
y = y.flatten().reshape(-1, 1)

k = np.zeros(x.shape)
q=0
while q < len(x):
    k[q] = loguniform.rvs(0.01, 100)
    if k[q] > 0.01 and k[q]< 100:
        q+=1

eps = 1e-1
task1 = {'field': np.concatenate([x, y, k], axis=1),
        'sources': {src: 100. for src in [465]},
        'sinks': [63],
        'dt': 0.000005,
        'boundary': torch.tensor([[-eps,-eps],
                                  [-eps, l + eps],
                                   [l + eps, l + eps],
                                    [l + eps, -eps]], requires_grad=False),
        'n_steps': 5000}

In [ ]:
scatter = plt.scatter(task1['field'][:, 0], task1['field'][:, 1], c=task1['field'][:, 2])
plt.title("Original permeability field")
plt.colorbar(scatter)

In [ ]:
ground_truth, ani = dynamics(task1['field'], sources=task1['sources'],
                             sinks=task1['sinks'], boundary=task1['boundary'],
                             dt=task1['dt'], save_animation=False, n_steps=task1['n_steps'])
plt.plot(ground_truth[0])


In [ ]:
set_seeds(41)
storage, coarsener = try_different_degrees_of_reduction(task1['field'], task1['boundary'], task1['dt'], task1['n_steps'],
                                                        reduction_degrees=[0.1], n_epochs = 900, lr = 0.03, sigmoid_weight = 1.0, phys_loss_weight = 100,
                                                        aggregating_function=KMeans, method='gnn', sources = task1['sources'], sinks = task1['sinks'], save_ani=False,
                                                        denseconv=False, num_layers=3,temperature=0.4,n_steps_val=0,warmup_steps=200,gatconv = True)


In [ ]:
result_competing_method = try_different_degrees_of_reduction(task1['field'], task1['boundary'], task1['dt'], task1['n_steps'], reduction_degrees=[0.1], n_epochs=300, aggregating_function=KMeans, method='points',
                                                             sources = task1['sources'], sinks = task1['sinks'],clamp_x = [0, 1.0], clamp_y = [0, 1.0], lr = 0.001,save_ani=False)